# ETL:  Exportaciones

## 0. Librerias

In [1]:
# Generales
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Funciones Snowflake
import funciones as snow_func

In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }

# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x0000027FA6C9C190>}


In [5]:
# Crear objeto de conexión
conn = session.connection

## 1. Base de exportaciones

In [6]:
# Solo se debe cambiar la ubicación del archivo
path_exportaciones = "C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/EXPORTACIONES/"
exportaciones_file = 'Base_Exportaciones_Colombianas.csv'

In [7]:
# Se descargó el Excel, se eliminaron las demás pestañas y solo se dejó la base de datos y exportó a CSV
# Se eligen los tipos de columna ya conocidos
df_exportaciones = pd.read_csv(path_exportaciones + exportaciones_file, sep=";", decimal=",",
                                dtype= {
                                    'Tipo' : str,
                                    'Cadena' : str,
                                    'Sector' : str,
                                    'Subsector' : str,
                                    'Posicion' : str,
                                    'Descripcion Posicion' : str,
                                    'Nit Exportador' : str,
                                    'Razon Social' : str,
                                    'Registrada NEO' : str,
                                    'Pais Destino' : str,
                                    'HUB' : str,
                                    'Continente' : str,
                                    'Zona Geografica' : str,
                                    'Tipo Acuerdo' : str,
                                    'Departamento Origen' : str,
                                    'Medio Transporte' : str,
                                    'Cadena Frío' : str,
                                    'Economia Naranja' : str,
                                    'TIPO*' : str,
                                    'CADENA*' : str,
                                    'SECTOR*' : str,
                                    'SUBSECTOR*' : str,
                                    '*DPTO MAS EXPORTA' : str,
                                    '2018 USD' : float,
                                    '2018 KG Neto' : float,
                                    '2019 USD' : float,
                                    '2019 KG Neto' : float,
                                    '2020 USD' : float,
                                    '2020 KG Neto' : float,
                                    '2021 USD' : float,
                                    '2021 KG Neto' : float,
                                    '2022 USD' : float,
                                    '2022 KG Neto' : float,
                                    '2023 USD' : float,
                                    '2023 KG Neto' : float,
                                    '2023 USD (Ene-Oct)' : float,
                                    '2023 KG Neto (Ene-Oct)' : float,
                                    '2024 USD (Ene-Oct)' : float,
                                    '2024 KG Neto (Ene-Oct)' : float
                                })
df_exportaciones.shape

# Crear insumo para validación en el paso # 8
df_insumo_validacion = df_exportaciones

In [8]:
# Identificar las columnas que no son de años
id_vars = [col for col in df_exportaciones.columns if not any(keyword in col for keyword in ['USD', 'KG Neto'])]
id_vars

['Tipo',
 'Cadena',
 'Sector',
 'Subsector',
 'Posicion',
 'Descripcion Posicion',
 'Nit Exportador',
 'Razon Social',
 'Registrada NEO',
 'Pais Destino',
 'HUB',
 'Continente',
 'Zona Geografica',
 "TLC'S",
 'Tipo Acuerdo',
 'Departamento Origen',
 'Medio Transporte',
 'Cadena Frío',
 'Economia Naranja',
 'TIPO*',
 'CADENA*',
 'SECTOR*',
 'SUBSECTOR*',
 '*DPTO MAS EXPORTA']

In [9]:
# Crear listas con las columnas que contienen valores (USD y KG Neto)
value_vars_usd = [col for col in df_exportaciones.columns if 'USD' in col]
value_vars_kg = [col for col in df_exportaciones.columns if 'KG Neto' in col]

In [10]:
value_vars_usd

['2018 USD',
 '2019 USD',
 '2020 USD',
 '2021 USD',
 '2022 USD',
 '2023 USD',
 '2023 USD (Ene-Oct)',
 '2024 USD (Ene-Oct)']

In [11]:
value_vars_kg

['2018 KG Neto',
 '2019 KG Neto',
 '2020 KG Neto',
 '2021 KG Neto',
 '2022 KG Neto',
 '2023 KG Neto',
 '2023 KG Neto (Ene-Oct)',
 '2024 KG Neto (Ene-Oct)']

In [12]:
# Convertir a formato long para USD
df_usd_long = df_exportaciones.melt(id_vars=id_vars, value_vars=value_vars_usd, var_name='Año', value_name='Valor USD')
df_usd_long['Año'] = df_usd_long['Año'].str.replace("USD", "")
df_usd_long['Año'] = df_usd_long['Año'].str.replace(" ", "")
df_usd_long = df_usd_long[df_usd_long['Valor USD'] > 0]
df_usd_long.head(5)

,Tipo,Cadena,Sector,Subsector,Posicion,Descripcion Posicion,Nit Exportador,Razon Social,Registrada NEO,Pais Destino,HUB,Continente,Zona Geografica,TLC'S,Tipo Acuerdo,Departamento Origen,Medio Transporte,Cadena Frío,Economia Naranja,TIPO*,CADENA*,SECTOR*,SUBSECTOR*,*DPTO MAS EXPORTA,Año,Valor USD
2,No Mineras,Metalmecánica y Otras Industrias,Artesanías,Cerámica,6914900000,Las demás manufacturas de cerámica.,9001271404,C.I.A MIGUEL CABALLERO SAS,Si,Afganistán,Norteamérica,Asia,Asia del Sur,Resto de países,Sin Acuerdo,Cundinamarca,Aéreo,No,No aplica,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,Bogotá,2018,278.0000000000
6,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,6307909000,Los demás artículos confeccionados,9001271404,C.I.A MIGUEL CABALLERO SAS,Si,Afganistán,Norteamérica,Asia,Asia del Sur,Resto de países,Sin Acuerdo,Cundinamarca,Aéreo,No,No aplica,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,Bogotá,2018,"1,903.0000000000"
20,No Mineras,Agroalimentos,Banano,Banano,0803901100,"Bananas o plátanos frescos del tipo ""cavendish valery""",9008378146,C.I. NOVA FOODS EXPORT S.A.S.,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Magdalena,Marítimo,Si,No aplica,No Mineras,Agroalimentos,Banano,Banano,Magdalena,2018,"16,200.0000000000"
22,No Mineras,Agroalimentos,Banano,Banano,0803901100,"Bananas o plátanos frescos del tipo ""cavendish valery""",9009348972,C.I BACCOTA S.A.S,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Antioquia,Marítimo,Si,No aplica,No Mineras,Agroalimentos,Banano,Banano,Antioquia,2018,"639,203.4000000000"
25,No Mineras,Agroalimentos,Café,Café verde,0901119000,"Los demás cafés sin tostar, sin descafeinar.",9001105941,OLAM AGRO COLOMBIA S.A.S,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Huila,Marítimo,No,No aplica,No Mineras,Agroalimentos,Café,Café verde,Huila,2018,"62,336.3000000000"


In [13]:
# Convertir a formato long para KG Neto
df_kg_long = df_exportaciones.melt(id_vars=id_vars, value_vars=value_vars_kg, var_name='Año', value_name='Peso KG')
df_kg_long['Año'] = df_kg_long['Año'].str.replace("KG Neto", "")
df_kg_long['Año'] = df_kg_long['Año'].str.replace(" ", "")
df_kg_long = df_kg_long[df_kg_long['Peso KG'] > 0]
df_kg_long.head(5)

,Tipo,Cadena,Sector,Subsector,Posicion,Descripcion Posicion,Nit Exportador,Razon Social,Registrada NEO,Pais Destino,HUB,Continente,Zona Geografica,TLC'S,Tipo Acuerdo,Departamento Origen,Medio Transporte,Cadena Frío,Economia Naranja,TIPO*,CADENA*,SECTOR*,SUBSECTOR*,*DPTO MAS EXPORTA,Año,Peso KG
2,No Mineras,Metalmecánica y Otras Industrias,Artesanías,Cerámica,6914900000,Las demás manufacturas de cerámica.,9001271404,C.I.A MIGUEL CABALLERO SAS,Si,Afganistán,Norteamérica,Asia,Asia del Sur,Resto de países,Sin Acuerdo,Cundinamarca,Aéreo,No,No aplica,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,Bogotá,2018,6.0000000000
6,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,6307909000,Los demás artículos confeccionados,9001271404,C.I.A MIGUEL CABALLERO SAS,Si,Afganistán,Norteamérica,Asia,Asia del Sur,Resto de países,Sin Acuerdo,Cundinamarca,Aéreo,No,No aplica,No Mineras,Sistema Moda,Textiles y confecciones,Otras confecciones,Bogotá,2018,12.5000000000
20,No Mineras,Agroalimentos,Banano,Banano,0803901100,"Bananas o plátanos frescos del tipo ""cavendish valery""",9008378146,C.I. NOVA FOODS EXPORT S.A.S.,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Magdalena,Marítimo,Si,No aplica,No Mineras,Agroalimentos,Banano,Banano,Magdalena,2018,"39,182.0000000000"
22,No Mineras,Agroalimentos,Banano,Banano,0803901100,"Bananas o plátanos frescos del tipo ""cavendish valery""",9009348972,C.I BACCOTA S.A.S,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Antioquia,Marítimo,Si,No aplica,No Mineras,Agroalimentos,Banano,Banano,Antioquia,2018,"1,407,807.0000000000"
25,No Mineras,Agroalimentos,Café,Café verde,0901119000,"Los demás cafés sin tostar, sin descafeinar.",9001105941,OLAM AGRO COLOMBIA S.A.S,Si,Albania,Europa,Europa,Europa del Sur,Resto de países,Sin Acuerdo,Huila,Marítimo,No,No aplica,No Mineras,Agroalimentos,Café,Café verde,Huila,2018,"19,954.0000000000"


In [14]:
# Para subir a Snowflake los nombres de las columnas no deben tener espacios, carácteres especiales, tildes o empezados por números
# Limpiar espacios y carácteres especiales

# USD
df_usd_long.columns = df_usd_long.columns.str.replace(' ', '_') 
df_usd_long.columns = df_usd_long.columns.str.replace('(', '') 
df_usd_long.columns = df_usd_long.columns.str.replace(')', '')
df_usd_long.columns = df_usd_long.columns.str.replace('-', '_')
df_usd_long.columns = df_usd_long.columns.str.replace("TLC'S", 'TLCS')

# KG
df_kg_long.columns = df_kg_long.columns.str.replace(' ', '_') 
df_kg_long.columns = df_kg_long.columns.str.replace('(', '') 
df_kg_long.columns = df_kg_long.columns.str.replace(')', '')
df_kg_long.columns = df_kg_long.columns.str.replace('-', '_')
df_kg_long.columns = df_kg_long.columns.str.replace("TLC'S", 'TLCS')

In [15]:
# Nuevos nombres de columnas (No pueden empezar con números ni tener * o tildes)

# USD
df_usd_long = df_usd_long.rename(columns={
    'TIPO*' : 'TIPO_ESTRELLA', 
    'CADENA*' : 'CADENA_ESTRELLA', 
    'SECTOR*' : 'SECTOR_ESTRELLA',
    'SUBSECTOR*' : 'SUBSECTOR_ESTRELLA', 
    '*DPTO_MAS_EXPORTA': 'DPTO_MAS_EXPORTA_ESTRELLA',
    'Cadena_Frío' : 'Cadena_Frio',
    'Año' : 'Year'
})

# KG
df_kg_long = df_kg_long.rename(columns={
    'TIPO*' : 'TIPO_ESTRELLA', 
    'CADENA*' : 'CADENA_ESTRELLA', 
    'SECTOR*' : 'SECTOR_ESTRELLA',
    'SUBSECTOR*' : 'SUBSECTOR_ESTRELLA', 
    '*DPTO_MAS_EXPORTA': 'DPTO_MAS_EXPORTA_ESTRELLA',
    'Cadena_Frío' : 'Cadena_Frio',
    'Año' : 'Year'
})

In [16]:
# Cambiar a mayúsculas
# USD
df_usd_long.columns = map(str.upper, df_usd_long.columns)
# KG
df_kg_long.columns = map(str.upper, df_kg_long.columns)

## 2. Parámetros (cambiar el mes)

In [17]:
# Crear un DataFrame que represente TEMP_PARAMETROS_CERRADO
TEMP_PARAMETROS_CERRADO = pd.DataFrame({
    'T_1_YEAR': ['2022'],
    'T_YEAR': ['2023']
})

# Mostrar el DataFrame resultante
TEMP_PARAMETROS_CERRADO

,T_1_YEAR,T_YEAR
0,2022,2023


In [18]:
# Crear un DataFrame que represente TEMP_PARAMETROS_CORRIDO
TEMP_PARAMETROS_CORRIDO = pd.DataFrame({
    'T_1_YEAR': ['2023(Ene-Oct)'],
    'T_YEAR': ['2024(Ene-Oct)']
})

# Mostrar el DataFrame resultante
TEMP_PARAMETROS_CORRIDO

,T_1_YEAR,T_YEAR
0,2023(Ene-Oct),2024(Ene-Oct)


## 3. Correlativa de Geografía

In [19]:
# Correlativa con la definición de Continentes
query_continentes = """
SELECT DISTINCT A.PAIS_LLAVE_EXPORTACIONES,
    B.REGION_NAME
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
LEFT JOIN DOCUMENTOS_COLOMBIA.GEOGRAFIA.CONTINENTES AS B ON A.REGION_NAME_EXPORTACIONES = B.REGION_NAME_EXPORTACIONES
WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL
ORDER BY 1 ASC;
"""
# Ejecutar
df_continentes = snow_func.snowflake_sql(conn, query_continentes)

In [20]:
# Correlativa con la definición de HUBS
query_hubs = """
	SELECT DISTINCT A.PAIS_LLAVE_EXPORTACIONES,
		B.NOMBRE_HUB
	FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
	LEFT JOIN DOCUMENTOS_COLOMBIA.GEOGRAFIA.HUBS AS B ON A.HUB_NAME_EXPORTACIONES = B.HUB_NAME_EXPORTACIONES
	WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL

UNION ALL

	SELECT A.PAIS_LLAVE_EXPORTACIONES,
		C.NOMBRE_HUB
	FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
	LEFT JOIN DOCUMENTOS_COLOMBIA.GEOGRAFIA.CARIBE_PAISES AS B ON A.M49_CODE = B.M49_CODE
	LEFT JOIN DOCUMENTOS_COLOMBIA.GEOGRAFIA.HUBS AS C ON B.ID_HUB = C.ID_HUB
	WHERE C.NOMBRE_HUB = 'Caribe';
"""
# Ejecutar
df_hubs = snow_func.snowflake_sql(conn, query_hubs)

In [21]:
# Correlativa con la definición de TLCs
query_tlcs = """
SELECT DISTINCT A.PAIS_LLAVE_EXPORTACIONES,
	A.NOMBRE_TLC
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL
ORDER BY 1 ASC;
"""
# Ejecutar
df_tlcs = snow_func.snowflake_sql(conn, query_tlcs)

In [22]:
# Se trae la tabla completa de geografía para la verificación de países
query_geografia = """
SELECT *
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL;
"""
# Ejecutar
df_verif = snow_func.snowflake_sql(conn, query_geografia)

In [23]:
# Lista con los dfs de geografía relevantes para el conteo de empresas
df_geo_list = {'CONTINENTES' : df_continentes, 
               'HUBS' : df_hubs, 
               'TLCS': df_tlcs}

## 3.1 Verificar completitud de países

In [24]:
# Elegir columnas de interés

# Correlativa
df_geo_paises_expo = df_verif[['PAIS_LLAVE_EXPORTACIONES', 'COUNTRY_OR_AREA']].drop_duplicates()

# Exportaciones USD
df_usd_long_paises_expo = df_usd_long[['PAIS_DESTINO']].drop_duplicates()

# Exportaciones KG
df_kg_long_paises_expo = df_kg_long[['PAIS_DESTINO']].drop_duplicates()


In [25]:
# Realizar unión

# USD
df_usd_long_paises_expo = df_usd_long_paises_expo.merge(df_geo_paises_expo, how='left', left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES')

# KG
df_kg_long_paises_expo = df_kg_long_paises_expo.merge(df_geo_paises_expo, how='left', left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES')

In [26]:
# No deben salir países de verdad nulos, solo agrupaciones internacionales o zonas francas
# En caso de que salgan países nulos, se deben agregar al modelo relacional
df_usd_long_paises_expo[df_usd_long_paises_expo['COUNTRY_OR_AREA'].isna()]

,PAIS_DESTINO,PAIS_LLAVE_EXPORTACIONES,COUNTRY_OR_AREA
191,ZF de Barranquilla,NaN,NaN
192,ZF de Bogotá,NaN,NaN
193,ZF de Candelaria Cartagena,NaN,NaN
194,ZF de Cartagena,NaN,NaN
195,ZF de Pacifico Cali,NaN,NaN
196,ZF de Palmaseca Cali,NaN,NaN
197,ZF de Rio negro Medellín,NaN,NaN
198,ZF Santander,NaN,NaN
199,ZFP de Tocancipá,NaN,NaN
200,ZFP Internacional de Pereira,NaN,NaN


In [27]:
# No deben salir países de verdad nulos, solo agrupaciones internacionales o zonas francas
# En caso de que salgan países nulos, se deben agregar al modelo relacional
df_kg_long_paises_expo[df_kg_long_paises_expo['COUNTRY_OR_AREA'].isna()]

,PAIS_DESTINO,PAIS_LLAVE_EXPORTACIONES,COUNTRY_OR_AREA
191,ZF de Barranquilla,NaN,NaN
192,ZF de Bogotá,NaN,NaN
193,ZF de Candelaria Cartagena,NaN,NaN
194,ZF de Cartagena,NaN,NaN
195,ZF de Pacifico Cali,NaN,NaN
196,ZF de Palmaseca Cali,NaN,NaN
197,ZF de Rio negro Medellín,NaN,NaN
198,ZF Santander,NaN,NaN
199,ZFP de Tocancipá,NaN,NaN
200,ZFP Internacional de Pereira,NaN,NaN


## 4. Crear totales de control

#### Funciones

In [28]:
def generar_totales(df, df_parametros, valor_col):
    """
    Genera un DataFrame con los totales agrupados para los años especificados en df_parametros.
    
    Parámetros:
    df (DataFrame): DataFrame original que contiene los datos a agrupar.
    df_parametros (DataFrame): DataFrame que contiene los años 'T_1_YEAR' y 'T_YEAR'.
    valor_col (str): Nombre de la columna que contiene los valores que se van a sumar.
    
    Retorna:
    DataFrame con los totales sumados.
    """
    # Sumar los valores para el año T_1_YEAR
    suma_t_1 = df[df['YEAR'] == df_parametros['T_1_YEAR'].iloc[0]][valor_col].sum()
    
    # Sumar los valores para el año T_YEAR
    suma_t = df[df['YEAR'] == df_parametros['T_YEAR'].iloc[0]][valor_col].sum()
    
    # Crear el DataFrame con los resultados
    df_total_usd = pd.DataFrame({
        'TABLA' : ['TOTAL'],
        'CATEGORIA': ['TOTAL'],
        'SUMA_USD_T_1': [suma_t_1],
        'SUMA_USD_T': [suma_t]
    })

    # Crear sumas de control
    suma_t_1_control =  df_total_usd['SUMA_USD_T_1']
    suma_t_control =  df_total_usd['SUMA_USD_T']
      
    return suma_t_1_control, suma_t_control

### Totales de control

In [29]:
# TOTALES USD Y KG
# Estos totales se calculan con la misma lógica al interior de las funciones de la siguiente sección para realizar la verificación.

# Cerrado
# USD TOTAL
suma_total_usd_t_1_control_cerrado, suma_total_usd_t_control_cerrado = generar_totales(df=df_usd_long, df_parametros=TEMP_PARAMETROS_CERRADO, valor_col='VALOR_USD')
# USD NME
suma_total_usd_t_1_nme_control_cerrado, suma_total_usd_t_nme_control_cerrado = generar_totales(df=df_usd_long[df_usd_long['TIPO']=='No Mineras'], df_parametros=TEMP_PARAMETROS_CERRADO, valor_col='VALOR_USD')
# PESO TOTAL
suma_total_kg_t_1_control_cerrado, suma_total_kg_t_control_cerrado = generar_totales(df=df_kg_long, df_parametros=TEMP_PARAMETROS_CERRADO, valor_col='PESO_KG')
# PESO MINERAS
suma_total_kg_t_1_mineras_control_cerrado, suma_total_kg_t_mineras_control_cerrado = generar_totales(df=df_kg_long[(df_kg_long['TIPO']=='Mineras') & (df_kg_long['MEDIO_TRANSPORTE'] != 'Otras vías')], df_parametros=TEMP_PARAMETROS_CERRADO, valor_col='PESO_KG')
# PESO NME
suma_total_kg_t_1_nme_control_cerrado, suma_total_kg_t_nme_control_cerrado = generar_totales(df=df_kg_long[(df_kg_long['TIPO']=='No Mineras') & (df_kg_long['MEDIO_TRANSPORTE'] != 'Otras vías')], df_parametros=TEMP_PARAMETROS_CERRADO, valor_col='PESO_KG')


# Corrido
# USD TOTAL
suma_total_usd_t_1_control_corrido, suma_total_usd_t_control_corrido = generar_totales(df=df_usd_long, df_parametros=TEMP_PARAMETROS_CORRIDO, valor_col='VALOR_USD')
# USD NME
suma_total_usd_t_1_nme_control_corrido, suma_total_usd_t_nme_control_corrido = generar_totales(df=df_usd_long[df_usd_long['TIPO']=='No Mineras'], df_parametros=TEMP_PARAMETROS_CORRIDO, valor_col='VALOR_USD')
# PESO TOTAL
suma_total_kg_t_1_control_corrido, suma_total_kg_t_control_corrido = generar_totales(df=df_kg_long, df_parametros=TEMP_PARAMETROS_CORRIDO, valor_col='PESO_KG')
# PESO MINERAS
suma_total_kg_t_1_mineras_control_corrido, suma_total_kg_t_mineras_control_corrido = generar_totales(df=df_kg_long[(df_kg_long['TIPO']=='Mineras') & (df_kg_long['MEDIO_TRANSPORTE'] != 'Otras vías')], df_parametros=TEMP_PARAMETROS_CORRIDO, valor_col='PESO_KG')
# PESO NME
suma_total_kg_t_1_nme_control_corrido, suma_total_kg_t_nme_control_corrido = generar_totales(df=df_kg_long[(df_kg_long['TIPO']=='No Mineras') & (df_kg_long['MEDIO_TRANSPORTE'] != 'Otras vías')], df_parametros=TEMP_PARAMETROS_CORRIDO, valor_col='PESO_KG')

## 5. Crear agrupaciones

#### Funciones

In [30]:
def generar_tabla_agrupada(df, agrupacion, unidad, tabla, categoria_col, valor_col, temp_parametros):
    """
    Genera una tabla agregada con métricas financieras de un DataFrame de exportaciones, agrupando por diferentes categorías.
    
    Optimizada para reducir el tiempo de ejecución filtrando previamente y usando operaciones vectorizadas.
    """
    
    # Filtrar los datos por los dos años antes de agrupar para evitar filtrado en cada agregación
    df_t1 = df[df['YEAR'] == temp_parametros['T_1_YEAR'].iloc[0]]
    df_t = df[df['YEAR'] == temp_parametros['T_YEAR'].iloc[0]]

    # Determinar las columnas de agrupación
    if categoria_col == unidad:
        group_cols = [unidad]
    else:
        group_cols = [categoria_col] + ([unidad] if unidad in df.columns else [])
    
    # Agrupar para cada año por separado
    suma_t1 = df_t1.groupby(group_cols)[valor_col].sum().reset_index(name='SUMA_USD_T_1')
    suma_t = df_t.groupby(group_cols)[valor_col].sum().reset_index(name='SUMA_USD_T')

    # Unir los resultados de los dos años
    df_agrupado = pd.merge(suma_t1, suma_t, on=group_cols, how='outer').fillna(0)

    # Agregar las columnas 'AGRUPACION' y 'TABLA'
    df_agrupado['AGRUPACION'] = agrupacion
    df_agrupado['TABLA'] = tabla

    # Agregar la columna 'UNIDAD'
    if unidad in df.columns:
        df_agrupado['UNIDAD'] = df_agrupado[unidad]
    else:
        df_agrupado['UNIDAD'] = unidad

    # Asignar la columna 'CATEGORIA'
    if categoria_col == unidad:
        df_agrupado['CATEGORIA'] = df_agrupado['UNIDAD']
    else:
        df_agrupado = df_agrupado.rename(columns={categoria_col: 'CATEGORIA'})

    # Calcular las diferencias absoluta y porcentual utilizando operaciones vectorizadas
    df_agrupado['DIFERENCIA_ABSOLUTA'] = df_agrupado['SUMA_USD_T'] - df_agrupado['SUMA_USD_T_1']
    df_agrupado['DIFERENCIA_PORCENTUAL'] = np.where(
        df_agrupado['SUMA_USD_T_1'] == 0,
        np.where(df_agrupado['SUMA_USD_T'] > 0, 100, np.where(df_agrupado['SUMA_USD_T'] == 0, 0, -100)),
        (df_agrupado['DIFERENCIA_ABSOLUTA'] / df_agrupado['SUMA_USD_T_1']) * 100
    )

    # Reordenar las columnas
    df_agrupado = df_agrupado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T', 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL']]

    return df_agrupado

In [31]:
def generar_tabla_totales(df, agrupacion, unidad, valor_col, temp_parametros):
    """
    Genera una tabla que contiene los totales de valores financieros agrupados por una unidad específica,
    o un total general si 'unidad' es una cadena estática. Calcula las diferencias absolutas y porcentuales
    entre dos períodos de tiempo definidos.

    Parameters:
    -----------
    df (pd.DataFrame): DataFrame con los datos a procesar.
    agrupacion (str): Nombre de la agrupación para la columna 'AGRUPACION'.
    unidad (str): Nombre de la columna a usar como 'UNIDAD', o un valor constante si es estático.
    valor_col (str): Nombre de la columna con los valores numéricos a sumar.
    temp_parametros (pd.DataFrame): DataFrame con los parámetros de tiempo, columnas 'T_1_YEAR' y 'T_YEAR'.

    Returns:
    --------
    pd.DataFrame: DataFrame con las columnas:
        - 'AGRUPACION': Nivel de agrupación.
        - 'UNIDAD': Nombre de la unidad o cadena estática.
        - 'TABLA': 'TOTAL'.
        - 'CATEGORIA': 'TOTAL'.
        - 'SUMA_USD_T_1': Suma de los valores para T_1_YEAR.
        - 'SUMA_USD_T': Suma de los valores para T_YEAR.
        - 'DIFERENCIA_ABSOLUTA': Diferencia entre T_YEAR y T_1_YEAR.
        - 'DIFERENCIA_PORCENTUAL': Cambio porcentual entre T_YEAR y T_1_YEAR.
    """
    
    # Filtrar de antemano los datos por los años T_1_YEAR y T_YEAR para mejorar eficiencia
    df_t_1 = df[df['YEAR'] == temp_parametros['T_1_YEAR'].iloc[0]]
    df_t = df[df['YEAR'] == temp_parametros['T_YEAR'].iloc[0]]

    # Si 'unidad' es una columna dinámica
    if unidad in df.columns:
        # Agrupar por 'unidad' y sumar los valores
        suma_t_1 = df_t_1.groupby(unidad)[valor_col].sum().reset_index(name='SUMA_USD_T_1')
        suma_t = df_t.groupby(unidad)[valor_col].sum().reset_index(name='SUMA_USD_T')
        
        # Combinar los resultados por unidad
        df_totales = pd.merge(suma_t_1, suma_t, on=unidad, how='outer').fillna(0)
        df_totales['UNIDAD'] = df_totales[unidad]
    else:
        # Si 'unidad' es un valor estático, calcular los totales sin agrupación
        suma_t_1 = df_t_1[valor_col].sum()
        suma_t = df_t[valor_col].sum()
        
        df_totales = pd.DataFrame({
            'AGRUPACION': [agrupacion],
            'UNIDAD': [unidad],
            'TABLA': ['TOTAL'],
            'CATEGORIA': ['TOTAL'],
            'SUMA_USD_T_1': [suma_t_1],
            'SUMA_USD_T': [suma_t]
        })

    # Agregar columnas 'AGRUPACION', 'TABLA', y 'CATEGORIA' si es una columna dinámica
    if unidad in df.columns:
        df_totales['AGRUPACION'] = agrupacion
        df_totales['TABLA'] = 'TOTAL'
        df_totales['CATEGORIA'] = 'TOTAL'

    # Reordenar las columnas
    df_totales = df_totales[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T']]

    # Calcular la diferencia absoluta y la diferencia porcentual usando operaciones vectorizadas
    df_totales['DIFERENCIA_ABSOLUTA'] = df_totales['SUMA_USD_T'] - df_totales['SUMA_USD_T_1']
    df_totales['DIFERENCIA_PORCENTUAL'] = np.where(
        (df_totales['SUMA_USD_T_1'] == 0) & (df_totales['SUMA_USD_T'] > 0), 100,
        np.where(
            (df_totales['SUMA_USD_T_1'] > 0) & (df_totales['SUMA_USD_T'] == 0), -100,
            np.where(
                (df_totales['SUMA_USD_T_1'] == 0) & (df_totales['SUMA_USD_T'] == 0), 0,
                (df_totales['DIFERENCIA_ABSOLUTA'] / df_totales['SUMA_USD_T_1']) * 100
            )
        )
    )

    return df_totales

In [32]:
def generar_tablas_completas_usd(df, df_control, agrupacion, valor_col, temp_parametros):
    """
    Genera un DataFrame completo con tablas agrupadas y totales para una agrupación específica. La función calcula
    los totales financieros de exportaciones, agrupa por diferentes categorías y valida los datos a través de sumas
    parciales y totales para verificar la integridad de la información.

    Parameters:
    df (pd.DataFrame): DataFrame de entrada que contiene los datos de exportaciones con columnas de valores y años preagrupados para velocidad.
    df_control (pd.DataFrame): DataFrame de entrada que contiene la base de exportaciones completa sin agrupar en formato long para crear totales de control.
    agrupacion (str): Nombre de la agrupación que define las unidades, tablas y categorías (ej. 'CONTINENTES', 'TLCS').
    valor_col (str): Nombre de la columna que contiene los valores a sumar (por ejemplo, 'VALOR_USD').
    temp_parametros (pd.DataFrame): DataFrame que contiene los parámetros temporales, con las columnas 'T_1_YEAR' 
                                    y 'T_YEAR', que definen los años a comparar.

    Returns:
    tuple: Retorna dos DataFrames:
        - df_resultado_filtrado: DataFrame con las tablas agrupadas y los totales.
        - df_validador_final: DataFrame que contiene los resultados de validación de las sumas parciales y totales.
    """
    
    # Definir nombres de las tablas y categorías para agrupar los datos
    tablas = ['CONTINENTE', 'DEPARTAMENTOS', 'PAIS', 'SECTORES', 'SUBSECTORES']
    categorias = ['CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'PAIS_DESTINO', 'SECTOR', 'SUBSECTOR']

    # Crear totales de control para validar la integridad de los datos
    # Suma total de exportaciones (USD) para todos los datos
    suma_total_usd_t_1_control, suma_total_usd_t_control = generar_totales(df=df_control, df_parametros=temp_parametros, valor_col='VALOR_USD')
    # Suma total de exportaciones (USD) para el tipo 'No Mineras'
    suma_total_usd_t_1_nme_control, suma_total_usd_t_nme_control = generar_totales(df=df_control[df_control['TIPO']=='No Mineras'], df_parametros=temp_parametros, valor_col='VALOR_USD')
    
    # Definir la columna que se usará como unidad de agrupación según el valor de 'agrupacion'
    if agrupacion == 'CONTINENTES':
        unidad = 'CONTINENTE'
    elif agrupacion == 'TLCS':
        unidad = 'TLCS'
    elif agrupacion == 'PAISES':
        unidad = 'PAIS_DESTINO'
    elif agrupacion == 'DEPARTAMENTOS':
        unidad = 'DEPARTAMENTO_ORIGEN'
    elif agrupacion == 'HUBS':
        unidad = 'HUB'
    elif agrupacion == 'COLOMBIA':
        unidad = 'COLOMBIA'
    else:
        raise ValueError("Agrupación no reconocida. Por favor, define los parámetros correspondientes.")

    # Inicializar DataFrames vacíos para almacenar los resultados finales y las validaciones
    df_resultado_final = pd.DataFrame()
    df_validador_final = pd.DataFrame()

    # Loop para generar tablas agrupadas por cada combinación de 'tabla' y 'categoria_col'
    df_nme = df[df['TIPO']=='No Mineras']
    for tabla, categoria_col in zip(tablas, categorias):
        print(f"Agrupación: {agrupacion}, Categoría: {categoria_col}")
        
        # Generar la tabla agrupada para la categoría actual
        df_agrupada = generar_tabla_agrupada(
            df=df_nme,
            agrupacion=agrupacion,
            unidad=unidad,
            tabla=tabla,
            categoria_col=categoria_col,
            valor_col=valor_col,
            temp_parametros=temp_parametros
        )
        
        # Concatenar los resultados al DataFrame final
        df_resultado_final = pd.concat([df_resultado_final, df_agrupada], ignore_index=True)

        # Validar los datos sumando las columnas 'SUMA_USD_T_1' y 'SUMA_USD_T' para la categoría actual
        df_validador = df_agrupada[df_agrupada['TABLA'] == tabla].groupby(['TABLA', 'CATEGORIA'])[['SUMA_USD_T_1', 'SUMA_USD_T']].sum().reset_index()
        
        # Sumar los valores parciales
        suma_parcial_t_1 = df_validador['SUMA_USD_T_1'].sum()
        suma_parcial_t = df_validador['SUMA_USD_T'].sum()

        # Definir el margen de tolerancia (en este caso, 1)
        tolerancia = 1
        # Comparación para la primera suma con tolerancia
        resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_usd_t_1_nme_control.iloc[0])) <= tolerancia
        # Comparación para la segunda suma con tolerancia
        resultado_t = abs(round(suma_parcial_t) - round(suma_total_usd_t_nme_control.iloc[0])) <= tolerancia
        
        # Mostrar los resultados de la validación
        print(resultado_t_1, resultado_t)
        
        # Guardar los resultados de la validación en un DataFrame
        df_resultados_validacion = pd.DataFrame({
            'AGRUPACION': [agrupacion],
            'UNIDAD': [unidad],
            'TABLA': [tabla],
            'CATEGORIA': [categoria_col],
            'SUMA_USD_T_1': [resultado_t_1],
            'SUMA_USD_T': [resultado_t]
        })
        
        # Concatenar las validaciones al DataFrame final de validación
        df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)

    # Generar la tabla de totales y agregarla al DataFrame final
    print(f"Agregando totales - Agrupación: {agrupacion}, Unidad: {unidad}")
    totales = generar_tabla_totales(
        df=df,
        agrupacion=agrupacion,
        unidad=unidad,
        valor_col=valor_col,
        temp_parametros=temp_parametros
    )
    df_resultado_final = pd.concat([df_resultado_final, totales], ignore_index=True)

    # Validar los datos de los totales
    suma_parcial_t_1 = totales['SUMA_USD_T_1'].sum()
    suma_parcial_t = totales['SUMA_USD_T'].sum()
    
    # Definir el margen de tolerancia (en este caso, 1)
    tolerancia = 1
    # Comparación para la primera suma con tolerancia
    resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_usd_t_1_control.iloc[0])) <= tolerancia
    # Comparación para la segunda suma con tolerancia
    resultado_t = abs(round(suma_parcial_t) - round(suma_total_usd_t_control.iloc[0])) <= tolerancia

    
    # Mostrar los resultados de la validación
    print(resultado_t_1, resultado_t)
    
    # Guardar los resultados de la validación para los totales
    df_resultados_validacion = pd.DataFrame({
        'AGRUPACION': [agrupacion],
        'UNIDAD': [unidad],
        'TABLA': ['TOTAL'],
        'CATEGORIA': ['TOTAL'],
        'SUMA_USD_T_1': [resultado_t_1],
        'SUMA_USD_T': [resultado_t]
    })
    df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)
   
    # Generar la tabla de tipos y agregarla al DataFrame final
    print(f"Agregando tipos - Agrupación: {agrupacion}, Unidad: {unidad}")
    # Definir nombres de las tablas y categorías para agrupar los datos
    tabla_tipo = 'TIPOS'
    categoria_tipo = 'TIPO'
    tipos = generar_tabla_agrupada(df=df, 
                           agrupacion=agrupacion, 
                           unidad=unidad, 
                           tabla=tabla_tipo, 
                           categoria_col=categoria_tipo, 
                           valor_col=valor_col, 
                           temp_parametros=temp_parametros) 
    df_resultado_final = pd.concat([df_resultado_final, tipos], ignore_index=True)
    # Validar los datos de los totales
    suma_parcial_t_1 = tipos['SUMA_USD_T_1'].sum()
    suma_parcial_t = tipos['SUMA_USD_T'].sum()
    
    # Definir el margen de tolerancia (en este caso, 1)
    tolerancia = 1
    # Comparación para la primera suma con tolerancia
    resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_usd_t_1_control.iloc[0])) <= tolerancia
    # Comparación para la segunda suma con tolerancia
    resultado_t = abs(round(suma_parcial_t) - round(suma_total_usd_t_control.iloc[0])) <= tolerancia

    # Mostrar los resultados de la validación
    print(resultado_t_1, resultado_t)
    
    # Guardar los resultados de la validación para los totales
    df_resultados_validacion = pd.DataFrame({
        'AGRUPACION': [agrupacion],
        'UNIDAD': [unidad],
        'TABLA': [tabla_tipo],
        'CATEGORIA': [categoria_tipo],
        'SUMA_USD_T_1': [resultado_t_1],
        'SUMA_USD_T': [resultado_t]
    })
    df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)

    # Eliminar filas donde los valores en ambas columnas de sumas sean cero
    condicion_1 = (df_resultado_final['SUMA_USD_T_1'] == 0) & (df_resultado_final['SUMA_USD_T'] == 0)
    df_resultado_filtrado = df_resultado_final[~condicion_1].reset_index(drop=True)

    # Retornar los DataFrames finales de agrupación y validación
    return df_resultado_filtrado, df_validador_final

In [33]:
def generar_tablas_completas_kg(df, df_control, agrupacion, valor_col, temp_parametros):
    """
    Genera un DataFrame completo con tablas agrupadas y totales para una agrupación específica. La función calcula
    los totales financieros de exportaciones, agrupa por diferentes categorías y valida los datos a través de sumas
    parciales y totales para verificar la integridad de la información.

    Parameters:
    df (pd.DataFrame): DataFrame de entrada que contiene los datos de exportaciones con columnas de valores y años preagrupados para velocidad.
    df_control (pd.DataFrame): DataFrame de entrada que contiene la base de exportaciones completa sin agrupar en formato long para crear totales de control.
    agrupacion (str): Nombre de la agrupación que define las unidades, tablas y categorías (ej. 'CONTINENTES', 'TLCS').
    valor_col (str): Nombre de la columna que contiene los valores a sumar (por ejemplo, 'VALOR_USD', 'PESO_KG').
    temp_parametros (pd.DataFrame): DataFrame que contiene los parámetros temporales, con las columnas 'T_1_YEAR' 
                                    y 'T_YEAR', que definen los años a comparar.

    Returns:
    tuple: Retorna dos DataFrames:
        - df_resultado_filtrado: DataFrame con las tablas agrupadas y los totales.
        - df_validador_final: DataFrame que contiene los resultados de validación de las sumas parciales y totales.
    """
    
    # Definir nombres de las tablas y categorías para agrupar los datos
    # Tablas de medios mineros
    tablas_mineros = ['MEDIO MINERAS', 'PAIS MINERAS']
    categorias_mineros = ['MEDIO_TRANSPORTE', 'PAIS_DESTINO']
    # Tablas de medios no mineros
    tablas_nme = ['MEDIO NO MINERAS', 'PAIS NO MINERAS']
    categorias_nme = ['MEDIO_TRANSPORTE', 'PAIS_DESTINO']

    # Crear totales de control para validar la integridad de los datos
    # Suma total de exportaciones (KG) para todos los datos
    suma_total_kg_t_1_control, suma_total_kg_t_control = generar_totales(df=df_control, df_parametros=temp_parametros, valor_col='PESO_KG')
    # Suma total de exportaciones (KG) para el tipo 'No Mineras'
    suma_total_kg_t_1_nme_control, suma_total_kg_t_nme_control = generar_totales(df=df_control[df_control['TIPO']=='No Mineras'], df_parametros=temp_parametros, valor_col='PESO_KG')
    # Suma total de exportaciones (KG) para el tipo 'Mineras'
    suma_total_kg_t_1_mineros_control, suma_total_kg_t_mineros_control = generar_totales(df=df_control[df_control['TIPO']=='Mineras'], df_parametros=temp_parametros, valor_col='PESO_KG')
    
    # Definir la columna que se usará como unidad de agrupación según el valor de 'agrupacion'
    if agrupacion == 'CONTINENTES':
        unidad = 'CONTINENTE'
    elif agrupacion == 'TLCS':
        unidad = 'TLCS'
    elif agrupacion == 'PAISES':
        unidad = 'PAIS_DESTINO'
    elif agrupacion == 'DEPARTAMENTOS':
        unidad = 'DEPARTAMENTO_ORIGEN'
    elif agrupacion == 'HUBS':
        unidad = 'HUB'
    elif agrupacion == 'COLOMBIA':
        unidad = 'COLOMBIA'
    else:
        raise ValueError("Agrupación no reconocida. Por favor, define los parámetros correspondientes.")

    # Inicializar DataFrames vacíos para almacenar los resultados finales y las validaciones
    df_resultado_final = pd.DataFrame()
    df_validador_final = pd.DataFrame()

    # Medios no mineras
    # Loop para generar tablas agrupadas por cada combinación de 'tabla' y 'categoria_col'
    df_nme = df[df['TIPO']=='No Mineras']
    for tabla, categoria_col in zip(tablas_nme, categorias_nme):
        print(f"Agrupación: {agrupacion}, Categoría: {categoria_col}")
        
        # Generar la tabla agrupada para la categoría actual
        df_agrupada = generar_tabla_agrupada(
            df=df_nme,
            agrupacion=agrupacion,
            unidad=unidad,
            tabla=tabla,
            categoria_col=categoria_col,
            valor_col=valor_col,
            temp_parametros=temp_parametros
        )
        
        # Concatenar los resultados al DataFrame final
        df_resultado_final = pd.concat([df_resultado_final, df_agrupada], ignore_index=True)

        # Validar los datos sumando las columnas 'SUMA_USD_T_1' y 'SUMA_USD_T' para la categoría actual
        df_validador = df_agrupada[df_agrupada['TABLA'] == tabla].groupby(['TABLA', 'CATEGORIA'])[['SUMA_USD_T_1', 'SUMA_USD_T']].sum().reset_index()
        
        # Sumar los valores parciales
        suma_parcial_t_1 = df_validador['SUMA_USD_T_1'].sum()
        suma_parcial_t = df_validador['SUMA_USD_T'].sum()

        # Definir el margen de tolerancia (en este caso, 1)
        tolerancia = 1
        # Comparación para la primera suma con tolerancia
        resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_kg_t_1_nme_control.iloc[0])) <= tolerancia
        # Comparación para la segunda suma con tolerancia
        resultado_t = abs(round(suma_parcial_t) - round(suma_total_kg_t_nme_control.iloc[0])) <= tolerancia
        
        # Mostrar los resultados de la validación
        print(resultado_t_1, resultado_t)
        
        # Guardar los resultados de la validación en un DataFrame
        df_resultados_validacion = pd.DataFrame({
            'AGRUPACION': [agrupacion],
            'UNIDAD': [unidad],
            'TABLA': [tabla],
            'CATEGORIA': [categoria_col],
            'SUMA_USD_T_1': [resultado_t_1],
            'SUMA_USD_T': [resultado_t]
        })
        
        # Concatenar las validaciones al DataFrame final de validación
        df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)

    # Medios mineras
    # Loop para generar tablas agrupadas por cada combinación de 'tabla' y 'categoria_col'
    df_mineros = df[df['TIPO']=='Mineras']
    for tabla, categoria_col in zip(tablas_mineros, categorias_mineros):
        print(f"Agrupación: {agrupacion}, Categoría: {categoria_col}")
        
        # Generar la tabla agrupada para la categoría actual
        df_agrupada = generar_tabla_agrupada(
            df=df_mineros,
            agrupacion=agrupacion,
            unidad=unidad,
            tabla=tabla,
            categoria_col=categoria_col,
            valor_col=valor_col,
            temp_parametros=temp_parametros
        )
        
        # Concatenar los resultados al DataFrame final
        df_resultado_final = pd.concat([df_resultado_final, df_agrupada], ignore_index=True)

        # Validar los datos sumando las columnas 'SUMA_USD_T_1' y 'SUMA_USD_T' para la categoría actual
        df_validador = df_agrupada[df_agrupada['TABLA'] == tabla].groupby(['TABLA', 'CATEGORIA'])[['SUMA_USD_T_1', 'SUMA_USD_T']].sum().reset_index()
        
        # Sumar los valores parciales
        suma_parcial_t_1 = df_validador['SUMA_USD_T_1'].sum()
        suma_parcial_t = df_validador['SUMA_USD_T'].sum()

        # Definir el margen de tolerancia (en este caso, 1)
        tolerancia = 1
        # Comparación para la primera suma con tolerancia
        resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_kg_t_1_mineros_control.iloc[0])) <= tolerancia
        # Comparación para la segunda suma con tolerancia
        resultado_t = abs(round(suma_parcial_t) - round(suma_total_kg_t_mineros_control.iloc[0])) <= tolerancia
        
        # Mostrar los resultados de la validación
        print(resultado_t_1, resultado_t)
        
        # Guardar los resultados de la validación en un DataFrame
        df_resultados_validacion = pd.DataFrame({
            'AGRUPACION': [agrupacion],
            'UNIDAD': [unidad],
            'TABLA': [tabla],
            'CATEGORIA': [categoria_col],
            'SUMA_USD_T_1': [resultado_t_1],
            'SUMA_USD_T': [resultado_t]
        })
        
        # Concatenar las validaciones al DataFrame final de validación
        df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)


    # Generar la tabla de totales y agregarla al DataFrame final
    print(f"Agregando totales - Agrupación: {agrupacion}, Unidad: {unidad}")
    totales = generar_tabla_totales(
        df=df,
        agrupacion=agrupacion,
        unidad=unidad,
        valor_col=valor_col,
        temp_parametros=temp_parametros
    )
    df_resultado_final = pd.concat([df_resultado_final, totales], ignore_index=True)

    # Validar los datos de los totales
    suma_parcial_t_1 = totales['SUMA_USD_T_1'].sum()
    suma_parcial_t = totales['SUMA_USD_T'].sum()
    
    # Definir el margen de tolerancia (en este caso, 1)
    tolerancia = 1
    # Comparación para la primera suma con tolerancia
    resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_kg_t_1_control.iloc[0])) <= tolerancia
    # Comparación para la segunda suma con tolerancia
    resultado_t = abs(round(suma_parcial_t) - round(suma_total_kg_t_control.iloc[0])) <= tolerancia

    
    # Mostrar los resultados de la validación
    print(resultado_t_1, resultado_t)
    
    # Guardar los resultados de la validación para los totales
    df_resultados_validacion = pd.DataFrame({
        'AGRUPACION': [agrupacion],
        'UNIDAD': [unidad],
        'TABLA': ['TOTAL'],
        'CATEGORIA': ['TOTAL'],
        'SUMA_USD_T_1': [resultado_t_1],
        'SUMA_USD_T': [resultado_t]
    })
    df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)
   
    # Generar la tabla de tipos y agregarla al DataFrame final
    print(f"Agregando tipos - Agrupación: {agrupacion}, Unidad: {unidad}")
    # Definir nombres de las tablas y categorías para agrupar los datos
    tabla_tipo = 'TIPOS'
    categoria_tipo = 'TIPO'
    tipos = generar_tabla_agrupada(df=df, 
                           agrupacion=agrupacion, 
                           unidad=unidad, 
                           tabla=tabla_tipo, 
                           categoria_col=categoria_tipo, 
                           valor_col=valor_col, 
                           temp_parametros=temp_parametros) 
    df_resultado_final = pd.concat([df_resultado_final, tipos], ignore_index=True)
    # Validar los datos de los totales
    suma_parcial_t_1 = tipos['SUMA_USD_T_1'].sum()
    suma_parcial_t = tipos['SUMA_USD_T'].sum()
    
    # Definir el margen de tolerancia (en este caso, 1)
    tolerancia = 1
    # Comparación para la primera suma con tolerancia
    resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_kg_t_1_control.iloc[0])) <= tolerancia
    # Comparación para la segunda suma con tolerancia
    resultado_t = abs(round(suma_parcial_t) - round(suma_total_kg_t_control.iloc[0])) <= tolerancia

    # Mostrar los resultados de la validación
    print(resultado_t_1, resultado_t)
    
    # Guardar los resultados de la validación para los totales
    df_resultados_validacion = pd.DataFrame({
        'AGRUPACION': [agrupacion],
        'UNIDAD': [unidad],
        'TABLA': [tabla_tipo],
        'CATEGORIA': [categoria_tipo],
        'SUMA_USD_T_1': [resultado_t_1],
        'SUMA_USD_T': [resultado_t]
    })
    df_validador_final = pd.concat([df_validador_final, df_resultados_validacion], ignore_index=True)

    # Eliminar filas donde los valores en ambas columnas de sumas sean cero
    condicion_1 = (df_resultado_final['SUMA_USD_T_1'] == 0) & (df_resultado_final['SUMA_USD_T'] == 0)
    df_resultado_filtrado = df_resultado_final[~condicion_1].reset_index(drop=True)

    # Retornar los DataFrames finales de agrupación y validación
    return df_resultado_filtrado, df_validador_final

In [34]:
def generar_agrupacion_nits(df, df_control, temp_parametros, agrupacion, unidad_col, tabla='EMPRESAS'):
    """
    Genera una tabla agregada por la unidad y agrupación especificada, optimizando la eficiencia al filtrar
    los años relevantes y usando operaciones vectorizadas. Además, genera un DataFrame de validación para comparar
    los totales calculados con un DataFrame de control.

    Parameters:
    -----------
    df (pd.DataFrame): DataFrame de entrada con los datos de exportaciones ya filtrados.
    df_control (pd.DataFrame): DataFrame de control que contiene la base de exportaciones completa para verificar totales.
    temp_parametros (pd.DataFrame): DataFrame con las columnas 'T_1_YEAR' y 'T_YEAR' que definen los años de comparación.
    agrupacion (str): Nombre de la agrupación, como 'CONTINENTES', 'DEPARTAMENTOS', 'PAISES' o 'COLOMBIA'.
    unidad_col (str): Nombre de la columna a utilizar como unidad, como 'CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'PAIS_DESTINO'.
    tabla (str): Nombre de la tabla (por defecto 'EMPRESAS').

    Returns:
    --------
    tuple: Retorna dos DataFrames:
        - df_resultado: pd.DataFrame: DataFrame con las columnas 'AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'RAZON_SOCIAL',
                                        'SECTOR_ESTRELLA', 'SUMA_USD_T_1', 'SUMA_USD_T'.
        - df_resultados_validacion: DataFrame que contiene los resultados de validación de las sumas parciales y totales.
    """

    # Crear totales de control
    suma_total_usd_t_1_control, suma_total_usd_t_control = generar_totales(df=df_control, df_parametros=temp_parametros, valor_col='VALOR_USD')

    # Obtener los años de comparación
    t_1_year = temp_parametros['T_1_YEAR'].iloc[0]
    t_year = temp_parametros['T_YEAR'].iloc[0]

    # Filtrar el DataFrame solo por los años relevantes (T_1_YEAR y T_YEAR)
    df_filtrado = df[df['YEAR'].isin([t_1_year, t_year])]

    # Utilizar operaciones vectorizadas en lugar de apply
    df_filtrado['SUMA_USD_T_1'] = np.where(df_filtrado['YEAR'] == t_1_year, df_filtrado['VALOR_USD'], 0)
    df_filtrado['SUMA_USD_T'] = np.where(df_filtrado['YEAR'] == t_year, df_filtrado['VALOR_USD'], 0)

    # Agrupación específica para 'COLOMBIA'
    if unidad_col == 'COLOMBIA':
        df_agrupado = df_filtrado.groupby(['NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA']).agg(
            SUMA_USD_T_1=('SUMA_USD_T_1', 'sum'),
            SUMA_USD_T=('SUMA_USD_T', 'sum')
        ).reset_index()
        df_agrupado['UNIDAD'] = 'COLOMBIA'  # Unidad constante para COLOMBIA

    else:
        # Agrupación por la unidad especificada
        df_agrupado = df_filtrado.groupby([unidad_col, 'NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA']).agg(
            SUMA_USD_T_1=('SUMA_USD_T_1', 'sum'),
            SUMA_USD_T=('SUMA_USD_T', 'sum')
        ).reset_index()
        df_agrupado['UNIDAD'] = df_agrupado[unidad_col]  # Agregar la unidad correspondiente

    # Agregar columnas AGRUPACION, TABLA y CATEGORIA
    df_agrupado['AGRUPACION'] = agrupacion
    df_agrupado['TABLA'] = tabla
    df_agrupado['CATEGORIA'] = df_agrupado['NIT_EXPORTADOR']

    # Reorganizar las columnas para el resultado final
    df_resultado = df_agrupado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', 'SUMA_USD_T_1', 'SUMA_USD_T']]

    # Verificar totales
    suma_parcial_t_1 = df_resultado['SUMA_USD_T_1'].sum()
    suma_parcial_t = df_resultado['SUMA_USD_T'].sum()

    # Definir el margen de tolerancia (en este caso, 1)
    tolerancia = 1

    # Comparación para la primera suma con tolerancia
    resultado_t_1 = abs(round(suma_parcial_t_1) - round(suma_total_usd_t_1_control.iloc[0])) <= tolerancia

    # Comparación para la segunda suma con tolerancia
    resultado_t = abs(round(suma_parcial_t) - round(suma_total_usd_t_control.iloc[0])) <= tolerancia

    # Mostrar los resultados de la validación
    print(resultado_t_1, resultado_t)

    # Guardar los resultados de la validación para los totales
    df_resultados_validacion = pd.DataFrame({
        'AGRUPACION': [agrupacion],
        'SUMA_USD_T_1': [resultado_t_1],
        'SUMA_USD_T': [resultado_t]
    })

    return df_resultado, df_resultados_validacion

In [35]:
def contar_empresas(df_long, agrupacion, temp_parametros, df_geografia, umbral=10000):
    """
    Función que cuenta el número de empresas (NITs únicos) que superan un umbral de exportaciones, agrupadas por diferentes categorías
    como CONTINENTES, DEPARTAMENTOS, PAISES, TLCS, HUBS, etc., para dos períodos de tiempo específicos (T_1_YEAR y T_YEAR).

    Parámetros:
    -----------
    df_long (pd.DataFrame): DataFrame con los datos de exportaciones. 
    agrupacion (str): Categoría de agrupación, puede ser 'HUBS', 'DEPARTAMENTOS', 'PAISES', 'TLCS', 'CONTINENTES', 'SUB_REGION' o 'COLOMBIA'.
    temp_parametros (pd.DataFrame): DataFrame con los parámetros temporales que contienen las columnas 'T_1_YEAR' y 'T_YEAR'.
    df_geografia (pd.DataFrame): DataFrame auxiliar con información geográfica adicional, necesario para algunas agrupaciones específicas.
    umbral (float): Valor mínimo de exportaciones en USD que debe superar una empresa para ser contada (default: 10000).

    Returns:
    --------
    pd.DataFrame: DataFrame con el número de empresas (NITs únicos) que superan el umbral, agrupadas por la categoría y los años.
    """

    # Aplicar filtros iniciales:
    # Filtrar por empresas "No Mineras" en las columnas 'TIPO' y 'TIPO_ESTRELLA'
    filtro_tipo = df_long['TIPO'] == "No Mineras"
    filtro_tipo_estrella = df_long['TIPO_ESTRELLA'] == "No Mineras"

    # Filtrar por las categorías específicas en la columna 'CADENA_ESTRELLA'
    filtro_cadena = df_long['CADENA_ESTRELLA'].isin([
        "Agroalimentos", 
        "Industrias 4.0", 
        "Metalmecánica y Otras Industrias", 
        "Químicos y Ciencias de la Vida", 
        "Sistema Moda"
    ])

    # Aplicar los filtros al DataFrame
    df_filtrado = df_long[filtro_tipo & filtro_tipo_estrella & filtro_cadena]
    
    # Definir un diccionario para mapear la agrupación seleccionada a la columna correspondiente
    column_map = {
        'HUBS': 'HUB',
        'DEPARTAMENTOS': 'DPTO_MAS_EXPORTA_ESTRELLA',
        'PAISES': 'PAIS_DESTINO',
        'TLCS': 'TLCS',
        'CONTINENTES': 'CONTINENTE',
        'COLOMBIA': 'NIT_EXPORTADOR',  # Para 'COLOMBIA', el conteo se hace por NIT_EXPORTADOR
        'SUB_REGION': ''  # SUB_REGION requiere un manejo especial más adelante
    }

    # Verificar si la agrupación seleccionada es válida
    if agrupacion not in column_map:
        raise ValueError(f"Agrupación {agrupacion} no es válida. Debe ser una de {list(column_map.keys())}")
    
    # Obtener la columna de agrupación adecuada
    agrupacion_col = column_map[agrupacion]

    # Filtrar registros donde NIT_EXPORTADOR no es -1
    df_filtrado = df_filtrado[df_filtrado['NIT_EXPORTADOR'] != '-1']

    # Caso especial para 'COLOMBIA': contar por NIT_EXPORTADOR y sumar VALOR_USD
    if agrupacion == 'COLOMBIA':
        conteo_empresas = pd.DataFrame()
        
        for year_col in ['T_1_YEAR', 'T_YEAR']:
            year = temp_parametros[year_col].iloc[0]  # Obtener el año actual

            # Filtrar por el año correspondiente
            df_year = df_filtrado[df_filtrado['YEAR'] == year]

            # Agrupar por NIT_EXPORTADOR y sumar VALOR_USD
            df_sumado = df_year.groupby('NIT_EXPORTADOR')['VALOR_USD'].sum().reset_index()

            # Filtrar empresas que superan el umbral de VALOR_USD
            df_filtrado_umbral = df_sumado[df_sumado['VALOR_USD'] > umbral]

            # Contar NIT_EXPORTADOR únicos
            conteo_nit_unicos = df_filtrado_umbral['NIT_EXPORTADOR'].nunique()

            # Guardar el resultado en el DataFrame final
            conteo_empresas[f'conteo_{year}'] = [conteo_nit_unicos]

        # Añadir la columna de agrupación con valor 'COLOMBIA'
        conteo_empresas['AGRUPACION'] = 'COLOMBIA'

        return conteo_empresas

    # Manejo especial para las agrupaciones 'CONTINENTES', 'SUB_REGION', 'TLCS', y 'HUBS'
    if agrupacion in ['CONTINENTES', 'SUB_REGION', 'TLCS', 'HUBS']:
        if agrupacion == 'CONTINENTES':
            # Unir con el DataFrame 'df_geografia' para obtener el código y nombre de la región
            df_filtrado = df_filtrado.merge(
                df_geografia[['PAIS_LLAVE_EXPORTACIONES', 'REGION_NAME']].drop_duplicates(),
                left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES', how='left'
            )
            agrupacion_col = 'REGION_NAME'
        
        elif agrupacion == 'SUB_REGION':
            # Unir con el DataFrame 'df_geografia' para obtener el código y nombre de la subregión
            df_filtrado = df_filtrado.merge(
                df_geografia[['PAIS_LLAVE_EXPORTACIONES', 'SUB_REGION_NAME']].drop_duplicates(),
                left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES', how='left'
            )
            agrupacion_col = 'SUB_REGION_NAME'

        elif agrupacion == 'TLCS':
            # Unir con el DataFrame 'df_geografia' para obtener el código y nombre del TLC
            df_filtrado = df_filtrado.merge(
                df_geografia[['PAIS_LLAVE_EXPORTACIONES', 'NOMBRE_TLC']].drop_duplicates(),
                left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES', how='left'
            )
            agrupacion_col = 'NOMBRE_TLC'

        elif agrupacion == 'HUBS':
            # Unir con el DataFrame 'CORR' para obtener el código y nombre del HUB
            df_filtrado = df_filtrado.merge(
                df_geografia[['PAIS_LLAVE_EXPORTACIONES', 'NOMBRE_HUB']].drop_duplicates(),
                left_on='PAIS_DESTINO', right_on='PAIS_LLAVE_EXPORTACIONES', how='left'
            )
            agrupacion_col = 'NOMBRE_HUB'

    # Filtrar por la columna de agrupación seleccionada (que no sea nula) (Departamentos y Países no requiere de un tratamiento especial)
    df_filtrado_agrupacion = df_filtrado[df_filtrado[agrupacion_col].notna()]

    # Crear un DataFrame vacío para almacenar los resultados
    conteo_empresas = pd.DataFrame()

    # Obtener todas las agrupaciones posibles en base a la columna de agrupación
    agrupaciones_completas = df_filtrado_agrupacion[agrupacion_col].dropna().unique()

    # Iterar por los años T_1_YEAR y T_YEAR
    for year_col in ['T_1_YEAR', 'T_YEAR']:
        year = temp_parametros[year_col].iloc[0]  # Obtener el año actual

        # Filtrar por el año actual
        df_year = df_filtrado_agrupacion[df_filtrado_agrupacion['YEAR'] == year]

        # Agrupar por la columna de agrupación y NIT_EXPORTADOR, y sumar VALOR_USD
        df_sumado = df_year.groupby([agrupacion_col, 'NIT_EXPORTADOR'])['VALOR_USD'].sum().reset_index()

        # Filtrar las empresas que superen el umbral de VALOR_USD
        df_filtrado_umbral = df_sumado[df_sumado['VALOR_USD'] > umbral]

        # Agrupar por la columna de agrupación y contar NITs únicos
        conteo_por_agrupacion = df_filtrado_umbral.groupby(agrupacion_col)['NIT_EXPORTADOR'].nunique().reset_index(name=f'conteo_{year}')

        # Asegurarse de que todas las agrupaciones posibles estén presentes (incluso con conteo 0)
        conteo_completo = pd.DataFrame(agrupaciones_completas, columns=[agrupacion_col]).merge(conteo_por_agrupacion, on=agrupacion_col, how='left').fillna(0)

        # Combinar los resultados de conteo con el DataFrame final
        if conteo_empresas.empty:
            conteo_empresas = conteo_completo
        else:
            conteo_empresas = pd.merge(conteo_empresas, conteo_completo, on=agrupacion_col, how='outer').fillna(0)

    # Convertir los conteos a enteros
    conteo_empresas[f'conteo_{temp_parametros["T_1_YEAR"].iloc[0]}'] = conteo_empresas[f'conteo_{temp_parametros["T_1_YEAR"].iloc[0]}'].astype(int)
    conteo_empresas[f'conteo_{temp_parametros["T_YEAR"].iloc[0]}'] = conteo_empresas[f'conteo_{temp_parametros["T_YEAR"].iloc[0]}'].astype(int)

    # Añadir la columna de AGRUPACION
    conteo_empresas['AGRUPACION'] = agrupacion

    # Cambiar el nombre de la primera columna a 'UNIDAD' si la agrupación no es 'COLOMBIA'
    if agrupacion != 'COLOMBIA':
        conteo_empresas.columns.values[0] = 'UNIDAD'

    return conteo_empresas


In [36]:
def contar_empresas_total(df_long, temp_parametros_cerrado, temp_parametros_corrido, df_list, umbral=10000):
    """
    Genera el conteo total de empresas que superan un umbral de exportaciones para diferentes agrupaciones, 
    tanto en periodos cerrados como corridos. Las agrupaciones incluyen Hubs, Departamentos, Países, TLCs, 
    Continentes, Colombia y Sub-Regiones.

    Parámetros:
    -----------
    df_long (pd.DataFrame): DataFrame de entrada que contiene los datos de exportaciones.
    temp_parametros_cerrado (pd.DataFrame): DataFrame con los parámetros temporales para el periodo cerrado.
    temp_parametros_corrido (pd.DataFrame): DataFrame con los parámetros temporales para el periodo corrido.
    df_list (list): Lista auxiliar con los dataframes de información geográfica (Hubs, TLCs, etc.).
    umbral (float): Valor mínimo de exportaciones en USD que debe superar una empresa para ser contada. (Por defecto 10000 USD).

    Returns:
    --------
    CONTEOCERRADO (pd.DataFrame): DataFrame que contiene los resultados del conteo de empresas para el periodo cerrado.
    CONTEOCORRIDO (pd.DataFrame): DataFrame que contiene los resultados del conteo de empresas para el periodo corrido.
    """
    
    # Lista de las agrupaciones que se van a procesar: Hubs, Departamentos, Países, TLCs, Continentes, Colombia y Sub-Regiones.
    agrupaciones = ['HUBS', 'DEPARTAMENTOS', 'PAISES', 'TLCS', 'CONTINENTES', 'COLOMBIA']
    
    # Inicializar listas para almacenar los resultados de conteo para los periodos cerrado y corrido.
    conteos_cerrado = []
    conteos_corrido = []

    # Iterar sobre cada agrupación para realizar el conteo de empresas tanto en el periodo cerrado como en el corrido.
    for agrupacion in agrupaciones:

        # Obtener correlativo dependiendo de la agrupación si es necesaria
        if agrupacion in ['HUBS', 'TLCS', 'CONTINENTES']:
            df_geografia = df_list[agrupacion]
        # Crear un df vacío si no requiere
        else:
            df_geografia = pd.DataFrame()
        
        # -------------------------------
        # Conteo para el periodo cerrado
        # -------------------------------
        # Llamar a la función contar_empresas para obtener el conteo del periodo cerrado según la agrupación actual.
        conteo_cerrado = contar_empresas(df_long, agrupacion, temp_parametros_cerrado, df_geografia, umbral)
        
        # Renombrar las columnas según la agrupación. Asegura que la columna 'UNIDAD' sea coherente con la agrupación.
        if agrupacion == 'HUBS':
            conteo_cerrado = conteo_cerrado.rename(columns={'NOMBRE_HUB': 'UNIDAD'})
        elif agrupacion == 'DEPARTAMENTOS':
            conteo_cerrado = conteo_cerrado.rename(columns={'DPTO_MAS_EXPORTA_ESTRELLA': 'UNIDAD'})
        elif agrupacion == 'PAISES':
            conteo_cerrado = conteo_cerrado.rename(columns={'PAIS_DESTINO': 'UNIDAD'})
        elif agrupacion == 'TLCS':
            conteo_cerrado = conteo_cerrado.rename(columns={'NOMBRE_TLC': 'UNIDAD'})
        elif agrupacion == 'CONTINENTES':
            conteo_cerrado = conteo_cerrado.rename(columns={'REGION_NAME': 'UNIDAD'})
        elif agrupacion == 'COLOMBIA':
            # Para Colombia, siempre se fija la unidad como 'COLOMBIA'.
            conteo_cerrado['UNIDAD'] = 'COLOMBIA'
        
        # Añadir el conteo del periodo cerrado a la lista de resultados.
        conteos_cerrado.append(conteo_cerrado)

        # -------------------------------
        # Conteo para el periodo corrido
        # -------------------------------
        # Llamar a la función contar_empresas para obtener el conteo del periodo corrido según la agrupación actual.
        conteo_corrido = contar_empresas(df_long, agrupacion, temp_parametros_corrido, df_geografia, umbral)
        
        # Renombrar las columnas según la agrupación. Igual que para el periodo cerrado, aseguramos que la columna 'UNIDAD' sea coherente.
        if agrupacion == 'HUBS':
            conteo_corrido = conteo_corrido.rename(columns={'NOMBRE_HUB': 'UNIDAD'})
        elif agrupacion == 'DEPARTAMENTOS':
            conteo_corrido = conteo_corrido.rename(columns={'DPTO_MAS_EXPORTA_ESTRELLA': 'UNIDAD'})
        elif agrupacion == 'PAISES':
            conteo_corrido = conteo_corrido.rename(columns={'PAIS_DESTINO': 'UNIDAD'})
        elif agrupacion == 'TLCS':
            conteo_corrido = conteo_corrido.rename(columns=({'NOMBRE_TLC': 'UNIDAD'}))
        elif agrupacion == 'CONTINENTES':
            conteo_corrido = conteo_corrido.rename(columns=({'REGION_NAME': 'UNIDAD'}))
        elif agrupacion == 'COLOMBIA':
            # Para Colombia, la unidad se fija como 'COLOMBIA' para el periodo corrido.
            conteo_corrido['UNIDAD'] = 'COLOMBIA'
        
        # Añadir el conteo del periodo corrido a la lista de resultados.
        conteos_corrido.append(conteo_corrido)

    # Concatenar los resultados de todas las agrupaciones para el periodo cerrado en un único DataFrame.
    CONTEOCERRADO = pd.concat(conteos_cerrado, axis=0)

    # Concatenar los resultados de todas las agrupaciones para el periodo corrido en un único DataFrame.
    CONTEOCORRIDO = pd.concat(conteos_corrido, axis=0)

    # Cambiar nombres
    CONTEOCERRADO.columns.values[1] = 'CONTEO_T_1'  # Cambiar el nombre de la columna 2
    CONTEOCERRADO.columns.values[2] = 'CONTEO_T'  # Cambiar el nombre de la columna 3

    # Cambiar nombres
    CONTEOCORRIDO.columns.values[1] = 'CONTEO_T_1'  # Cambiar el nombre de la columna 2
    CONTEOCORRIDO.columns.values[2] = 'CONTEO_T'  # Cambiar el nombre de la columna 3

    # Retornar los DataFrames finales para los periodos cerrado y corrido.
    return CONTEOCERRADO, CONTEOCORRIDO

In [37]:
def agregar_columnas_crecimiento(df, col_t_1, col_t):
    """
    Agrega las columnas de diferencia absoluta y crecimiento porcentual a un DataFrame, 
    calculando la diferencia entre las columnas proporcionadas.

    Parameters:
    df (pd.DataFrame): DataFrame de entrada que contiene las columnas con valores a comparar.
    col_t_1 (str): Nombre de la columna que representa los valores de un año anterior (T-1).
    col_t (str): Nombre de la columna que representa los valores del año actual (T).

    Returns:
    pd.DataFrame: El DataFrame con las columnas adicionales de 'DIFERENCIA_ABSOLUTA' y 'DIFERENCIA_PORCENTUAL'.
    """
    
    # Calcular la diferencia absoluta entre col_t y col_t_1
    df['DIFERENCIA_ABSOLUTA'] = df[col_t] - df[col_t_1]
    
    # Calcular la diferencia porcentual con una lógica de control para evitar divisiones por 0
    df['DIFERENCIA_PORCENTUAL'] = df.apply(
        lambda row: 100 if row[col_t_1] == 0 and row[col_t] > 0 else
                    -100 if row[col_t_1] > 0 and row[col_t] == 0 else
                    0 if row[col_t_1] == 0 and row[col_t] == 0 else
                    ((row[col_t] - row[col_t_1]) / row[col_t_1]) * 100,
        axis=1
    )
    
    return df


In [38]:
def procesar_df_por_agrupacion(df_input, correlativa, agrupacion, columna_correlativa, columna_renombrada):
    """
    Procesa un DataFrame de exportaciones agrupado por una unidad específica (ej. países), uniendo con una tabla correlativa, 
    sumando los valores y agregando columnas de diferencias.

    Parámetros:
    -----------
    df_input (pd.DataFrame): DataFrame de entrada que contiene las exportaciones ya agrupadas por diferentes categorías.
    correlativa (pd.DataFrame): Tabla correlativa que se utilizará para hacer el 'merge' con la columna correlativa.
    agrupacion (str): Nombre de la agrupación que se va a agregar a la columna 'AGRUPACION'.
    columna_correlativa (str): Nombre de la columna en la tabla correlativa para unir con el DataFrame de entrada.
    columna_renombrada (str): Nombre de la columna en el DataFrame correlativo que se renombrará a 'UNIDAD'.

    Retorna:
    --------
    pd.DataFrame: DataFrame procesado con las sumas de exportaciones, diferencias agregadas y agrupado por la unidad y categoría.
    """
    
    # Filtrar los datos para que solo contengan la agrupación seleccionada ('PAISES' en este caso).
    df_insumo = df_input[df_input['AGRUPACION'] == 'PAISES']
    
    # Realizar una unión (merge) con la tabla correlativa usando una relación 'left' entre la columna 'UNIDAD' del DataFrame de entrada 
    # y la columna especificada por 'columna_correlativa' en la tabla correlativa.
    df_resultado = df_insumo.merge(correlativa, how='left', left_on=['UNIDAD'], right_on=[columna_correlativa])
    
    # Seleccionar únicamente las columnas que son relevantes para el análisis, incluyendo la columna correlativa que se renombrará.
    df_resultado = df_resultado[['TABLA', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T', columna_renombrada]]
    
    # Renombrar la columna correlativa seleccionada a 'UNIDAD' para estandarizar los nombres de las columnas.
    df_resultado = df_resultado.rename(columns={columna_renombrada: 'UNIDAD'})
    
    # Añadir la columna 'AGRUPACION' al DataFrame con el valor pasado como argumento en la función.
    df_resultado['AGRUPACION'] = agrupacion
    
    # Reorganizar las columnas del DataFrame en un orden específico: AGRUPACION, UNIDAD, TABLA, CATEGORIA, SUMA_USD_T_1 y SUMA_USD_T.
    df_resultado = df_resultado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T']]
    
    # Agrupar los datos por las columnas 'AGRUPACION', 'UNIDAD', 'TABLA' y 'CATEGORIA', y sumar los valores de 'SUMA_USD_T_1' y 'SUMA_USD_T'.
    df_resultado = df_resultado.groupby(['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA'], as_index=False).agg({
        'SUMA_USD_T_1': 'sum',
        'SUMA_USD_T': 'sum'
    })
    
    # Agregar columnas de diferencias calculando el crecimiento entre las columnas 'SUMA_USD_T_1' y 'SUMA_USD_T'.
    df_resultado = agregar_columnas_crecimiento(df=df_resultado, col_t_1='SUMA_USD_T_1', col_t='SUMA_USD_T')
    
    # Retornar el DataFrame procesado con las sumas y diferencias de exportaciones.
    return df_resultado


In [39]:
# Función principal que crea la tabla consolidada para HUBS, TLCS y CONTINENTES
def crear_df_st_categorias(df_st_categorias, corrhubs, corrtlcs, corrcont):
    """
    Crea un DataFrame consolidado que incluye los datos procesados para HUBS, TLCs y CONTINENTES, 
    uniendo las correlativas correspondientes y eliminando las filas con sumas iguales a 0.

    Parámetros:
    -----------
    df_st_categorias (pd.DataFrame): DataFrame de entrada con las categorías base.
    corrhubs (pd.DataFrame): DataFrame con las correlativas de HUBS.
    corrtlcs (pd.DataFrame): DataFrame con las correlativas de TLCs.
    corrcont (pd.DataFrame): DataFrame con las correlativas de CONTINENTES.

    Retorna:
    --------
    pd.DataFrame: DataFrame consolidado con las categorías base y las correlativas de HUBS, TLCs y CONTINENTES.
    """
    
    # Procesar HUBS
    df_st_hubs = procesar_df_por_agrupacion(
        df_input=df_st_categorias,         # DataFrame de entrada con las categorías
        correlativa=corrhubs,              # Correlativa específica para HUBS
        agrupacion='HUBS',                 # Nombre de la agrupación que se va a asignar
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='NOMBRE_HUB'    # Columna que se va a renombrar a 'UNIDAD'
    )

    # Procesar TLCs
    df_st_tlcs = procesar_df_por_agrupacion(
        df_input=df_st_categorias,         # DataFrame de entrada con las categorías
        correlativa=corrtlcs,              # Correlativa específica para TLCs
        agrupacion='TLCS',                 # Nombre de la agrupación que se va a asignar
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='NOMBRE_TLC'    # Columna que se va a renombrar a 'UNIDAD'
    )

    # Procesar CONTINENTES
    df_st_cont = procesar_df_por_agrupacion(
        df_input=df_st_categorias,         # DataFrame de entrada con las categorías
        correlativa=corrcont,              # Correlativa específica para CONTINENTES
        agrupacion='CONTINENTES',          # Nombre de la agrupación que se va a asignar
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='REGION_NAME'   # Columna que se va a renombrar a 'UNIDAD'
    )

    # Concatenar los resultados de las tres agrupaciones (categorías base, HUBS, TLCs, CONTINENTES)
    df_st_completo = pd.concat([df_st_hubs, df_st_tlcs, df_st_cont])
    
    # Eliminar las filas donde tanto SUMA_USD_T_1 como SUMA_USD_T son iguales a 0
    condicion_1 = (df_st_completo['SUMA_USD_T_1'] == 0) & (df_st_completo['SUMA_USD_T'] == 0)
    df_st_completo = df_st_completo[~condicion_1].reset_index(drop=True)  # Resetear el índice después de eliminar filas
    
    return df_st_completo

In [40]:
def procesar_df_por_agrupacion_peso(df_input, correlativa, agrupacion, columna_correlativa, columna_renombrada):
    """
    Procesa un DataFrame de entrada, filtrando por la agrupación 'PAISES', uniendo con una tabla correlativa y 
    calculando las sumas y diferencias de peso para los períodos T_1 y T.

    Parámetros:
    -----------
    df_input (pd.DataFrame): DataFrame de entrada que contiene los datos de exportación, incluyendo los pesos.
    correlativa (pd.DataFrame): DataFrame correlativo que se unirá con df_input basado en una columna correlativa.
    agrupacion (str): Nombre de la agrupación a agregar al DataFrame final, como 'HUBS', 'TLCs', o 'CONTINENTES'.
    columna_correlativa (str): Nombre de la columna en el DataFrame correlativa para realizar la unión.
    columna_renombrada (str): Nombre de la columna que se renombrará a 'UNIDAD' después de la unión.

    Retorna:
    --------
    pd.DataFrame: DataFrame procesado, con las sumas de peso para T_1 y T, y con las columnas de crecimiento agregado.
    """
    
    # Filtrar el DataFrame de entrada para mantener solo las filas con la agrupación 'PAISES'
    df_insumo = df_input[df_input['AGRUPACION'] == 'PAISES']
    
    # Realizar la unión del DataFrame filtrado con la tabla correlativa, usando 'UNIDAD' y la columna correlativa como claves
    df_resultado = df_insumo.merge(correlativa, how='left', left_on=['UNIDAD'], right_on=[columna_correlativa])
    
    # Seleccionar únicamente las columnas relevantes del DataFrame resultante tras la unión
    df_resultado = df_resultado[['TABLA', 'CATEGORIA', 'SUMA_PESO_T_1', 'SUMA_PESO_T', columna_renombrada]]
    
    # Renombrar la columna correlativa a 'UNIDAD' para estandarizar el nombre en el DataFrame resultante
    df_resultado = df_resultado.rename(columns={columna_renombrada: 'UNIDAD'})
    
    # Agregar una nueva columna 'AGRUPACION' con el valor especificado en los parámetros de la función
    df_resultado['AGRUPACION'] = agrupacion
    
    # Reordenar las columnas del DataFrame para asegurar que estén en el orden correcto
    df_resultado = df_resultado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_PESO_T_1', 'SUMA_PESO_T']]
    
    # Agrupar los datos por las columnas 'AGRUPACION', 'UNIDAD', 'TABLA' y 'CATEGORIA', sumando los pesos de T_1 y T
    df_resultado = df_resultado.groupby(['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA'], as_index=False).agg({
        'SUMA_PESO_T_1': 'sum',  # Sumar los pesos de T_1
        'SUMA_PESO_T': 'sum'     # Sumar los pesos de T
    })
    
    # Agregar columnas adicionales calculando la diferencia y crecimiento porcentual entre T_1 y T 
    df_resultado = agregar_columnas_crecimiento(df=df_resultado, col_t_1='SUMA_PESO_T_1', col_t='SUMA_PESO_T' )
    
    # Devolver el DataFrame procesado con las columnas de sumas y crecimiento
    return df_resultado

In [41]:
# Función principal que crea la tabla consolidada para HUBS, TLCS y CONTINENTES, basada en el peso
def crear_df_st_categorias_peso(df_st_categorias, corrhubs, corrtlcs, corrcont):
    """
    Procesa el DataFrame `df_st_categorias` para consolidar los datos de peso en las categorías de HUBS, TLCS y CONTINENTES.
    La función realiza las operaciones de agrupación y suma de pesos para las categorías seleccionadas y luego elimina
    las filas donde ambos valores de peso son cero.

    Parámetros:
    -----------
    df_st_categorias (pd.DataFrame): DataFrame con los datos de entrada que contienen las categorías y pesos.
    corrhubs (pd.DataFrame): DataFrame de correlación que contiene la correspondencia de HUBS con los países.
    corrtlcs (pd.DataFrame): DataFrame de correlación que contiene la correspondencia de TLCs con los países.
    corrcont (pd.DataFrame): DataFrame de correlación que contiene la correspondencia de continentes con los países.

    Retorna:
    --------
    pd.DataFrame: DataFrame consolidado con las categorías de HUBS, TLCS y CONTINENTES, con las sumas de peso por categoría.
    """
    
    # Procesar HUBS: filtrar y procesar el DataFrame por la agrupación de HUBS
    df_st_hubs = procesar_df_por_agrupacion_peso(
        df_input=df_st_categorias,           # DataFrame de entrada
        correlativa=corrhubs,                # DataFrame de correlación para HUBS
        agrupacion='HUBS',                   # Nombre de la agrupación (HUBS)
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='NOMBRE_HUB'      # Columna a renombrar como 'UNIDAD'
    )

    # Procesar TLCS: filtrar y procesar el DataFrame por la agrupación de TLCs
    df_st_tlcs = procesar_df_por_agrupacion_peso(
        df_input=df_st_categorias,           # DataFrame de entrada
        correlativa=corrtlcs,                # DataFrame de correlación para TLCs
        agrupacion='TLCS',                   # Nombre de la agrupación (TLCS)
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='NOMBRE_TLC'      # Columna a renombrar como 'UNIDAD'
    )

    # Procesar CONTINENTES: filtrar y procesar el DataFrame por la agrupación de continentes
    df_st_cont = procesar_df_por_agrupacion_peso(
        df_input=df_st_categorias,           # DataFrame de entrada
        correlativa=corrcont,                # DataFrame de correlación para continentes
        agrupacion='CONTINENTES',            # Nombre de la agrupación (CONTINENTES)
        columna_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Columna correlativa para la unión
        columna_renombrada='REGION_NAME'     # Columna a renombrar como 'UNIDAD'
    )

    # Concatenar los DataFrames procesados para HUBS, TLCs y CONTINENTES
    df_st_completo = pd.concat([df_st_hubs, df_st_tlcs, df_st_cont])

    # Eliminar las filas donde las sumas de peso en T_1 y T sean iguales a cero
    condicion_1 = (df_st_completo['SUMA_PESO_T_1'] == 0) & (df_st_completo['SUMA_PESO_T'] == 0)
    df_st_completo = df_st_completo[~condicion_1].reset_index(drop=True)  # Filtrar y restablecer los índices

    # Devolver el DataFrame completo procesado
    return df_st_completo

In [42]:
def procesar_agrupacion(df_insumo, df_correlativo, llave_correlativa, nombre_columna, agrupacion):
    """
    Procesa un DataFrame para realizar una agrupación específica basada en una tabla correlativa y un conjunto de columnas de interés.
    
    Parámetros:
    -----------
    df_insumo : pd.DataFrame
        DataFrame que contiene los datos base que se van a agrupar.
    df_correlativo : pd.DataFrame
        DataFrame correlativo que contiene la información adicional necesaria para unir con 'df_insumo'.
    llave_correlativa : str
        Nombre de la columna en 'df_correlativo' que se utilizará como llave para la unión.
    nombre_columna : str
        Nombre de la columna que se utilizará como la nueva 'UNIDAD' en el DataFrame final.
    agrupacion : str
        Nombre de la agrupación que se agregará a la columna 'AGRUPACION' en el DataFrame final.
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con los datos agrupados por las columnas 'AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA',
        y las sumas de 'SUMA_USD_T_1' y 'SUMA_USD_T'.
    """
    
    # Realiza un merge (unión) entre 'df_insumo' y 'df_correlativo' utilizando 'UNIDAD' en 'df_insumo' y 'llave_correlativa' en 'df_correlativo'
    df_merged = df_insumo.merge(df_correlativo, how='left', left_on=['UNIDAD'], right_on=[llave_correlativa])

    # Selecciona las columnas de interés del DataFrame resultante de la unión
    df_merged = df_merged[['TABLA', 'CATEGORIA', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', 'SUMA_USD_T_1', 'SUMA_USD_T', nombre_columna]]

    # Renombra la columna 'nombre_columna' a 'UNIDAD' para estandarizar el nombre
    df_merged = df_merged.rename(columns={nombre_columna: 'UNIDAD'})

    # Agrega la columna 'AGRUPACION' con el valor proporcionado en el parámetro 'agrupacion'
    df_merged['AGRUPACION'] = agrupacion

    # Reorganiza las columnas en el orden deseado
    df_merged = df_merged[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', 'SUMA_USD_T_1', 'SUMA_USD_T']]

    # Agrupa los datos por las columnas clave y suma los valores de 'SUMA_USD_T_1' y 'SUMA_USD_T'
    df_merged = df_merged.groupby(
        ['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA'], 
        as_index=False
    ).agg({
        'SUMA_USD_T_1': 'sum',  # Sumar los valores de 'SUMA_USD_T_1'
        'SUMA_USD_T': 'sum'  # Sumar los valores de 'SUMA_USD_T'
    })

    # Retorna el DataFrame final con los datos procesados y agrupados
    return df_merged

In [43]:
def procesar_df_nit(df_input, corrhubs, corrtlcs, corrcont):
    """
    Procesa un DataFrame de entrada para realizar agrupaciones por HUBS, TLCs y CONTINENTES, uniendo con tablas correlativas
    y calculando las sumas de valores financieros. Luego elimina las filas con valores nulos y agrega columnas de crecimiento.

    Parámetros:
    -----------
    df_input : pd.DataFrame
        DataFrame de entrada que contiene los datos de exportaciones por NIT (identificación tributaria).
    corrhubs : pd.DataFrame
        DataFrame correlativo que contiene información sobre HUBs.
    corrtlcs : pd.DataFrame
        DataFrame correlativo que contiene información sobre TLCs (Tratados de Libre Comercio).
    corrcont : pd.DataFrame
        DataFrame correlativo que contiene información sobre continentes.
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame consolidado que incluye las agrupaciones por HUBs, TLCs y continentes, con columnas de sumas de USD y crecimiento.
    """
    
    # Crear DataFrame para HUBS realizando la unión y procesamiento con la tabla correlativa de HUBs
    df_st_hubs = procesar_agrupacion(
        df_insumo=df_input,
        df_correlativo=corrhubs,
        llave_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Llave para unir con la tabla correlativa
        nombre_columna='NOMBRE_HUB',  # Columna que se renombrará como 'UNIDAD'
        agrupacion='HUBS'  # Agrupación de HUBS
    )

    # Crear DataFrame para TLCs realizando la unión y procesamiento con la tabla correlativa de TLCs
    df_st_tlcs = procesar_agrupacion(
        df_insumo=df_input,
        df_correlativo=corrtlcs,
        llave_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Llave para unir con la tabla correlativa
        nombre_columna='NOMBRE_TLC',  # Columna que se renombrará como 'UNIDAD'
        agrupacion='TLCS'  # Agrupación de TLCs
    )

    # Crear DataFrame para CONTINENTES realizando la unión y procesamiento con la tabla correlativa de continentes
    df_st_cont = procesar_agrupacion(
        df_insumo=df_input,
        df_correlativo=corrcont,
        llave_correlativa='PAIS_LLAVE_EXPORTACIONES',  # Llave para unir con la tabla correlativa
        nombre_columna='REGION_NAME',  # Columna que se renombrará como 'UNIDAD'
        agrupacion='CONTINENTES'  # Agrupación de continentes
    )

    # Concatenar los DataFrames generados para HUBS, TLCs y CONTINENTES en un único DataFrame
    SD_ST_NIT = pd.concat([df_st_hubs, df_st_tlcs, df_st_cont], ignore_index=True)

    # Eliminar filas donde tanto 'SUMA_USD_T_1' como 'SUMA_USD_T' sean cero
    condicion_1 = (SD_ST_NIT['SUMA_USD_T_1'] == 0) & (SD_ST_NIT['SUMA_USD_T'] == 0)
    SD_ST_NIT = SD_ST_NIT[~condicion_1].reset_index(drop=True)  # Se eliminan esas filas y se resetea el índice

    # Agregar columnas de crecimiento calculadas entre los valores de 'SUMA_USD_T_1' y 'SUMA_USD_T' utilizando una función externa
    SD_ST_NIT = agregar_columnas_crecimiento(df=SD_ST_NIT, col_t_1='SUMA_USD_T_1', col_t='SUMA_USD_T')

    # Retornar el DataFrame final con las agrupaciones y cálculos de crecimiento
    return SD_ST_NIT

### 4.1 ST_CATEGORIAS_CERRADO

In [44]:
# Preparación (Preagrupación para asegurar velocidad)
df_insumo_minero_nme = df_usd_long.groupby(['CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'PAIS_DESTINO', 'SECTOR', 'SUBSECTOR', 'TIPO', 'YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})
df_insumo_control_usd = df_usd_long.groupby(['TIPO', 'YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})

In [45]:
# Definir los parámetros
AGRUPACIONES = ['COLOMBIA', 'PAISES', 'DEPARTAMENTOS']
VALOR_COL = 'VALOR_USD'
TEMP_PARAMETROS = TEMP_PARAMETROS_CERRADO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_CATEGORIAS_CERRADO = pd.DataFrame()
DF_ST_CATEGORIAS_CERRADO_VALIDACION = pd.DataFrame()

# Loop para ejecutar la función en cada agrupación y concatenar los resultados
for AGRUPACION in AGRUPACIONES:
    # Ejecutar la función para cada agrupación
    df_resultado, df_validacion = generar_tablas_completas_usd(
        df=df_insumo_minero_nme,
        df_control = df_insumo_control_usd,
        agrupacion=AGRUPACION,
        valor_col=VALOR_COL,
        temp_parametros=TEMP_PARAMETROS
    )
    
    # Concatenar los resultados de cada agrupación
    DF_ST_CATEGORIAS_CERRADO = pd.concat([DF_ST_CATEGORIAS_CERRADO, df_resultado], ignore_index=True)
    DF_ST_CATEGORIAS_CERRADO_VALIDACION = pd.concat([DF_ST_CATEGORIAS_CERRADO_VALIDACION, df_validacion], ignore_index=True)

Agrupación: COLOMBIA, Categoría: CONTINENTE
True True
Agrupación: COLOMBIA, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: SECTOR
True True
Agrupación: COLOMBIA, Categoría: SUBSECTOR
True True
Agregando totales - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agregando tipos - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agrupación: PAISES, Categoría: CONTINENTE
True True
Agrupación: PAISES, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: PAISES, Categoría: SECTOR
True True
Agrupación: PAISES, Categoría: SUBSECTOR
True True
Agregando totales - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agregando tipos - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: CONTINENTE
True True
Agrupación: DEPARTAMENTOS, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True

In [46]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_CATEGORIAS_CERRADO_VALIDACION[(DF_ST_CATEGORIAS_CERRADO_VALIDACION['SUMA_USD_T_1'] == False) | (DF_ST_CATEGORIAS_CERRADO_VALIDACION['SUMA_USD_T'] == False)]

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T


In [47]:
# Crear agrupaciones de hubs, tlcs, continentes
df_hubs_tlcs_continentes = crear_df_st_categorias(df_st_categorias=DF_ST_CATEGORIAS_CERRADO, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [48]:
# Crear tabla ST_CATEGORIAS_CERRADO
DF_ST_CATEGORIAS_CERRADO = pd.concat([DF_ST_CATEGORIAS_CERRADO, df_hubs_tlcs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_CATEGORIAS_CERRADO['SUMA_USD_T_1'] == 0) & (DF_ST_CATEGORIAS_CERRADO['SUMA_USD_T'] == 0)
DF_ST_CATEGORIAS_CERRADO = DF_ST_CATEGORIAS_CERRADO[~condicion_1].reset_index(drop=True)

### 4.2 ST_CATEGORIAS_CORRIDO

In [49]:
# Preparación (Preagrupación para asegurar velocidad)
df_insumo_minero_nme = df_usd_long.groupby(['CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'PAIS_DESTINO', 'SECTOR', 'SUBSECTOR', 'TIPO', 'YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})
df_insumo_control_usd = df_usd_long.groupby(['TIPO', 'YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})

In [50]:
# Definir los parámetros
AGRUPACIONES = ['COLOMBIA', 'PAISES', 'DEPARTAMENTOS']
VALOR_COL = 'VALOR_USD'
TEMP_PARAMETROS = TEMP_PARAMETROS_CORRIDO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_CATEGORIAS_CORRIDO = pd.DataFrame()
DF_ST_CATEGORIAS_CORRIDO_VALIDACION = pd.DataFrame()

# Loop para ejecutar la función en cada agrupación y concatenar los resultados
for AGRUPACION in AGRUPACIONES:
    # Ejecutar la función para cada agrupación
    df_resultado, df_validacion = generar_tablas_completas_usd(
        df=df_insumo_minero_nme,
        df_control = df_insumo_control_usd,
        agrupacion=AGRUPACION,
        valor_col=VALOR_COL,
        temp_parametros=TEMP_PARAMETROS
    )
    
    # Concatenar los resultados de cada agrupación
    DF_ST_CATEGORIAS_CORRIDO = pd.concat([DF_ST_CATEGORIAS_CORRIDO, df_resultado], ignore_index=True)
    DF_ST_CATEGORIAS_CORRIDO_VALIDACION = pd.concat([DF_ST_CATEGORIAS_CORRIDO_VALIDACION, df_validacion], ignore_index=True)

Agrupación: COLOMBIA, Categoría: CONTINENTE
True True
Agrupación: COLOMBIA, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: SECTOR
True True
Agrupación: COLOMBIA, Categoría: SUBSECTOR
True True
Agregando totales - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agregando tipos - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agrupación: PAISES, Categoría: CONTINENTE
True True
Agrupación: PAISES, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: PAISES, Categoría: SECTOR
True True
Agrupación: PAISES, Categoría: SUBSECTOR
True True
Agregando totales - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agregando tipos - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: CONTINENTE
True True
Agrupación: DEPARTAMENTOS, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True

In [51]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_CATEGORIAS_CORRIDO_VALIDACION[(DF_ST_CATEGORIAS_CORRIDO_VALIDACION['SUMA_USD_T_1'] == False) | (DF_ST_CATEGORIAS_CORRIDO_VALIDACION['SUMA_USD_T'] == False)]

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_USD_T_1,SUMA_USD_T


In [52]:
# Crear agrupaciones de hubs, tlcs, continentes
df_hubs_tlcs_continentes = crear_df_st_categorias(df_st_categorias=DF_ST_CATEGORIAS_CORRIDO, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [53]:
# Crear tabla ST_CATEGORIAS_CORRIDO
DF_ST_CATEGORIAS_CORRIDO = pd.concat([DF_ST_CATEGORIAS_CORRIDO, df_hubs_tlcs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_CATEGORIAS_CORRIDO['SUMA_USD_T_1'] == 0) & (DF_ST_CATEGORIAS_CORRIDO['SUMA_USD_T'] == 0)
DF_ST_CATEGORIAS_CORRIDO = DF_ST_CATEGORIAS_CORRIDO[~condicion_1].reset_index(drop=True)

### 4.3 ST_CATEGORIAS_PESO_CERRADO

In [54]:
# Preparación (Preagrupación para asegurar velocidad)
df_insumo_minero_nme = df_kg_long.groupby(['PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'MEDIO_TRANSPORTE', 'TIPO', 'YEAR'], as_index=False).agg({'PESO_KG': 'sum'})
df_insumo_control_kg = df_kg_long.groupby(['TIPO', 'YEAR'], as_index=False).agg({'PESO_KG': 'sum'})

In [55]:
# Definir los parámetros
AGRUPACIONES = ['COLOMBIA', 'PAISES', 'DEPARTAMENTOS']
VALOR_COL = 'PESO_KG'
TEMP_PARAMETROS = TEMP_PARAMETROS_CERRADO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_CATEGORIAS_PESO_CERRADO = pd.DataFrame()
DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION = pd.DataFrame()

# Loop para ejecutar la función en cada agrupación y concatenar los resultados
for AGRUPACION in AGRUPACIONES:
    # Ejecutar la función para cada agrupación
    df_resultado, df_validacion = generar_tablas_completas_kg(
        df=df_insumo_minero_nme,
        df_control = df_insumo_control_kg,
        agrupacion=AGRUPACION,
        valor_col=VALOR_COL,
        temp_parametros=TEMP_PARAMETROS
    )
    
    # Concatenar los resultados de cada agrupación
    DF_ST_CATEGORIAS_PESO_CERRADO = pd.concat([DF_ST_CATEGORIAS_PESO_CERRADO, df_resultado], ignore_index=True)
    DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION = pd.concat([DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION, df_validacion], ignore_index=True)

# Cambiar nombres de usd a kg ya que se usa la misma función que las categorias de usd
DF_ST_CATEGORIAS_PESO_CERRADO = DF_ST_CATEGORIAS_PESO_CERRADO.rename(columns={'SUMA_USD_T_1': 'SUMA_PESO_T_1', 'SUMA_USD_T': 'SUMA_PESO_T'})
DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION = DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION.rename(columns={'SUMA_USD_T_1': 'SUMA_PESO_T_1', 'SUMA_USD_T': 'SUMA_PESO_T'})

Agrupación: COLOMBIA, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agregando totales - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agregando tipos - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agrupación: PAISES, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: PAISES, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agregando totales - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agregando tipos - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True True
Agregando totale

In [56]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION[(DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION['SUMA_PESO_T_1'] == False) | (DF_ST_CATEGORIAS_PESO_CERRADO_VALIDACION['SUMA_PESO_T'] == False)]

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_PESO_T_1,SUMA_PESO_T


In [57]:
# Crear agrupaciones de hubs, tlcs, continentes
df_hubs_tlcs_continentes = crear_df_st_categorias_peso(df_st_categorias=DF_ST_CATEGORIAS_PESO_CERRADO, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [58]:
# Crear tabla ST_CATEGORIAS_PESO_CERRADO
DF_ST_CATEGORIAS_PESO_CERRADO = pd.concat([DF_ST_CATEGORIAS_PESO_CERRADO, df_hubs_tlcs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_CATEGORIAS_PESO_CERRADO['SUMA_PESO_T_1'] == 0) & (DF_ST_CATEGORIAS_PESO_CERRADO['SUMA_PESO_T'] == 0)
DF_ST_CATEGORIAS_PESO_CERRADO = DF_ST_CATEGORIAS_PESO_CERRADO[~condicion_1].reset_index(drop=True)

### 4.4 ST_CATEGORIAS_PESO_CORRIDO

In [59]:
# Preparación (Preagrupación para asegurar velocidad)
df_insumo_minero_nme = df_kg_long.groupby(['PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'MEDIO_TRANSPORTE', 'TIPO', 'YEAR'], as_index=False).agg({'PESO_KG': 'sum'})
df_insumo_control_kg = df_kg_long.groupby(['TIPO', 'YEAR'], as_index=False).agg({'PESO_KG': 'sum'})

In [60]:
# Definir los parámetros
AGRUPACIONES = ['COLOMBIA', 'PAISES', 'DEPARTAMENTOS']
VALOR_COL = 'PESO_KG'
TEMP_PARAMETROS = TEMP_PARAMETROS_CORRIDO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_CATEGORIAS_PESO_CORRIDO = pd.DataFrame()
DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION = pd.DataFrame()

# Loop para ejecutar la función en cada agrupación y concatenar los resultados
for AGRUPACION in AGRUPACIONES:
    # Ejecutar la función para cada agrupación
    df_resultado, df_validacion = generar_tablas_completas_kg(
        df=df_insumo_minero_nme,
        df_control = df_insumo_control_kg,
        agrupacion=AGRUPACION,
        valor_col=VALOR_COL,
        temp_parametros=TEMP_PARAMETROS
    )
    
    # Concatenar los resultados de cada agrupación
    DF_ST_CATEGORIAS_PESO_CORRIDO = pd.concat([DF_ST_CATEGORIAS_PESO_CORRIDO, df_resultado], ignore_index=True)
    DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION = pd.concat([DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION, df_validacion], ignore_index=True)

# Cambiar nombres de usd a kg ya que se usa la misma función que las categorias de usd
DF_ST_CATEGORIAS_PESO_CORRIDO = DF_ST_CATEGORIAS_PESO_CORRIDO.rename(columns={'SUMA_USD_T_1': 'SUMA_PESO_T_1', 'SUMA_USD_T': 'SUMA_PESO_T'})
DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION = DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION.rename(columns={'SUMA_USD_T_1': 'SUMA_PESO_T_1', 'SUMA_USD_T': 'SUMA_PESO_T'})

Agrupación: COLOMBIA, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: COLOMBIA, Categoría: PAIS_DESTINO
True True
Agregando totales - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agregando tipos - Agrupación: COLOMBIA, Unidad: COLOMBIA
True True
Agrupación: PAISES, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: PAISES, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agregando totales - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agregando tipos - Agrupación: PAISES, Unidad: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True True
Agrupación: DEPARTAMENTOS, Categoría: MEDIO_TRANSPORTE
True True
Agrupación: DEPARTAMENTOS, Categoría: PAIS_DESTINO
True True
Agregando totale

In [61]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION[(DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION['SUMA_PESO_T_1'] == False) | (DF_ST_CATEGORIAS_PESO_CORRIDO_VALIDACION['SUMA_PESO_T'] == False)]

,AGRUPACION,UNIDAD,TABLA,CATEGORIA,SUMA_PESO_T_1,SUMA_PESO_T


In [62]:
# Crear agrupaciones de hubs, tlcs, continentes
df_hubs_tlcs_continentes = crear_df_st_categorias_peso(df_st_categorias=DF_ST_CATEGORIAS_PESO_CORRIDO, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [63]:
# Crear tabla ST_CATEGORIAS_PESO_CORRIDO
DF_ST_CATEGORIAS_PESO_CORRIDO = pd.concat([DF_ST_CATEGORIAS_PESO_CORRIDO, df_hubs_tlcs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_CATEGORIAS_PESO_CORRIDO['SUMA_PESO_T_1'] == 0) & (DF_ST_CATEGORIAS_PESO_CORRIDO['SUMA_PESO_T'] == 0)
DF_ST_CATEGORIAS_PESO_CORRIDO = DF_ST_CATEGORIAS_PESO_CORRIDO[~condicion_1].reset_index(drop=True)

### 4.5 ST_NIT_CERRADO

In [64]:
# Preparación (Preagrupación para asegurar velocidad)
# df_insumo_nits = df_usd_long[(df_usd_long['TIPO'] == 'No Mineras') & (df_usd_long['TIPO_ESTRELLA'] == 'No Mineras') & (~df_usd_long['NIT_EXPORTADOR'].isin(['-1']))]
df_insumo_nits = df_usd_long[(df_usd_long['TIPO'] == 'No Mineras')]
df_insumo_nits = df_insumo_nits.groupby(['YEAR', 'NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', 'CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'HUB', 'PAIS_DESTINO'], as_index=False).agg({'VALOR_USD': 'sum'})
df_insumo_nits = df_insumo_nits[df_insumo_nits['VALOR_USD']>0]
# Base de control
df_control_nits = df_insumo_nits.groupby(['YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})

In [65]:
# Definir los casos de agrupación y sus respectivas columnas de unidad
casos_agrupacion = [
    {'agrupacion': 'DEPARTAMENTOS', 'unidad_col': 'DEPARTAMENTO_ORIGEN'},
    {'agrupacion': 'PAISES', 'unidad_col': 'PAIS_DESTINO'},
    {'agrupacion': 'COLOMBIA', 'unidad_col': 'COLOMBIA'}  # Unidad constante
]
TEMP_PARAMETROS = TEMP_PARAMETROS_CERRADO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_NIT_CERRADO = pd.DataFrame()
DF_ST_NIT_CERRADO_VALIDACION = pd.DataFrame()

# Para cada caso de agrupación, ejecutar la función y concatenar los resultados
for caso in casos_agrupacion:
    agrupacion = caso['agrupacion']
    unidad_col = caso['unidad_col']
    print(f"Agrupación: {agrupacion}, Categoría: {unidad_col}")
    
    # Llamar a la función con los parámetros correspondientes
    df_resultado, df_validacion = generar_agrupacion_nits(
        df=df_insumo_nits,                     # DataFrame de exportaciones filtrado
        df_control=df_control_nits,            # DataFrame de control
        temp_parametros=TEMP_PARAMETROS,       # DataFrame con T_1_YEAR y T_YEAR
        agrupacion=agrupacion,                 # Agrupación actual (ej. 'CONTINENTES', 'DEPARTAMENTOS')
        unidad_col=unidad_col,                 # Columna de la unidad (ej. 'CONTINENTE', 'DEPARTAMENTO_ORIGEN')
        tabla='EMPRESAS'                       # Nombre de la tabla (por defecto 'EMPRESAS')
    )
    
    # Concatenar los resultados para este caso con el DataFrame final
    DF_ST_NIT_CERRADO = pd.concat([DF_ST_NIT_CERRADO, df_resultado], ignore_index=True)
    DF_ST_NIT_CERRADO_VALIDACION = pd.concat([DF_ST_NIT_CERRADO_VALIDACION, df_validacion], ignore_index=True)

Agrupación: DEPARTAMENTOS, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: COLOMBIA
True True


In [66]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_NIT_CERRADO_VALIDACION[(DF_ST_NIT_CERRADO_VALIDACION['SUMA_USD_T_1'] == False) | (DF_ST_NIT_CERRADO_VALIDACION['SUMA_USD_T'] == False)]

,AGRUPACION,SUMA_USD_T_1,SUMA_USD_T


In [67]:
# Crear df de insumo para TLCS y HUBS
df_insumo_tlcs_hubs = DF_ST_NIT_CERRADO[DF_ST_NIT_CERRADO['AGRUPACION'] == 'PAISES']

In [68]:
# Crear agrupaciones de hubs, tlcs, continentes
df_nit_tlcs_hubs_continentes = procesar_df_nit(df_input=df_insumo_tlcs_hubs, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [69]:
# Crear tabla ST_NIT_CERRADO
DF_ST_NIT_CERRADO = pd.concat([DF_ST_NIT_CERRADO, df_nit_tlcs_hubs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_NIT_CERRADO['SUMA_USD_T_1'] == 0) & (DF_ST_NIT_CERRADO['SUMA_USD_T'] == 0)
DF_ST_NIT_CERRADO = DF_ST_NIT_CERRADO[~condicion_1].reset_index(drop=True)
# Agregar crecimiento
DF_ST_NIT_CERRADO = agregar_columnas_crecimiento(df=DF_ST_NIT_CERRADO, col_t_1='SUMA_USD_T_1', col_t='SUMA_USD_T')
# Eliminar repetidos
DF_ST_NIT_CERRADO = DF_ST_NIT_CERRADO.drop_duplicates()

### 4.5 ST_NIT_CORRIDO

In [70]:
# Preparación (Preagrupación para asegurar velocidad)
# df_insumo_nits = df_usd_long[(df_usd_long['TIPO'] == 'No Mineras') & (df_usd_long['TIPO_ESTRELLA'] == 'No Mineras') & (~df_usd_long['NIT_EXPORTADOR'].isin(['-1']))]
df_insumo_nits = df_usd_long[(df_usd_long['TIPO'] == 'No Mineras')]
df_insumo_nits = df_insumo_nits.groupby(['YEAR', 'NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', 'CONTINENTE', 'DEPARTAMENTO_ORIGEN', 'HUB', 'PAIS_DESTINO'], as_index=False).agg({'VALOR_USD': 'sum'})
df_insumo_nits = df_insumo_nits[df_insumo_nits['VALOR_USD']>0]
# Base de control
df_control_nits = df_insumo_nits.groupby(['YEAR'], as_index=False).agg({'VALOR_USD': 'sum'})

In [71]:
# Definir los casos de agrupación y sus respectivas columnas de unidad
casos_agrupacion = [
    {'agrupacion': 'DEPARTAMENTOS', 'unidad_col': 'DEPARTAMENTO_ORIGEN'},
    {'agrupacion': 'PAISES', 'unidad_col': 'PAIS_DESTINO'},
    {'agrupacion': 'COLOMBIA', 'unidad_col': 'COLOMBIA'}  # Unidad constante
]
TEMP_PARAMETROS = TEMP_PARAMETROS_CORRIDO 

# Inicializar DataFrames vacíos para almacenar los resultados finales
DF_ST_NIT_CORRIDO = pd.DataFrame()
DF_ST_NIT_CORRIDO_VALIDACION = pd.DataFrame()

# Para cada caso de agrupación, ejecutar la función y concatenar los resultados
for caso in casos_agrupacion:
    agrupacion = caso['agrupacion']
    unidad_col = caso['unidad_col']
    print(f"Agrupación: {agrupacion}, Categoría: {unidad_col}")
    
    # Llamar a la función con los parámetros correspondientes
    df_resultado, df_validacion = generar_agrupacion_nits(
        df=df_insumo_nits,                     # DataFrame de exportaciones filtrado
        df_control=df_control_nits,            # DataFrame de control
        temp_parametros=TEMP_PARAMETROS,       # DataFrame con T_1_YEAR y T_YEAR
        agrupacion=agrupacion,                 # Agrupación actual (ej. 'CONTINENTES', 'DEPARTAMENTOS')
        unidad_col=unidad_col,                 # Columna de la unidad (ej. 'CONTINENTE', 'DEPARTAMENTO_ORIGEN')
        tabla='EMPRESAS'                       # Nombre de la tabla (por defecto 'EMPRESAS')
    )
    
    # Concatenar los resultados para este caso con el DataFrame final
    DF_ST_NIT_CORRIDO = pd.concat([DF_ST_NIT_CORRIDO, df_resultado], ignore_index=True)
    DF_ST_NIT_CORRIDO_VALIDACION = pd.concat([DF_ST_NIT_CORRIDO_VALIDACION, df_validacion], ignore_index=True)

Agrupación: DEPARTAMENTOS, Categoría: DEPARTAMENTO_ORIGEN
True True
Agrupación: PAISES, Categoría: PAIS_DESTINO
True True
Agrupación: COLOMBIA, Categoría: COLOMBIA
True True


In [72]:
# Elegir observaciones con falsos en el df de validación, si no hay resultado, el proceso no tiene errores.
DF_ST_NIT_CORRIDO_VALIDACION[(DF_ST_NIT_CORRIDO_VALIDACION['SUMA_USD_T_1'] == False) | (DF_ST_NIT_CORRIDO_VALIDACION['SUMA_USD_T'] == False)]

,AGRUPACION,SUMA_USD_T_1,SUMA_USD_T


In [73]:
# Crear df de insumo para TLCS y HUBS
df_insumo_tlcs_hubs = DF_ST_NIT_CORRIDO[DF_ST_NIT_CORRIDO['AGRUPACION'] == 'PAISES']

In [74]:
# Crear agrupaciones de hubs, tlcs, continentes
df_nit_tlcs_hubs_continentes = procesar_df_nit(df_input=df_insumo_tlcs_hubs, corrhubs=df_hubs, corrtlcs=df_tlcs, corrcont=df_continentes)

In [75]:
# Crear tabla ST_NIT_CORRIDO
DF_ST_NIT_CORRIDO = pd.concat([DF_ST_NIT_CORRIDO, df_nit_tlcs_hubs_continentes])
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (DF_ST_NIT_CORRIDO['SUMA_USD_T_1'] == 0) & (DF_ST_NIT_CORRIDO['SUMA_USD_T'] == 0)
DF_ST_NIT_CORRIDO = DF_ST_NIT_CORRIDO[~condicion_1].reset_index(drop=True)
# Agregar crecimiento
DF_ST_NIT_CORRIDO = agregar_columnas_crecimiento(df=DF_ST_NIT_CORRIDO, col_t_1='SUMA_USD_T_1', col_t='SUMA_USD_T')
# Eliminar repetidos
DF_ST_NIT_CORRIDO = DF_ST_NIT_CORRIDO.drop_duplicates()

### 4.6 ST_CONTEO_CERRADO y ST_CONTEO_CORRIDO

In [76]:
# Realizar conteos:
ST_CONTEO_CERRADO, ST_CONTEO_CORRIDO = contar_empresas_total(df_long=df_usd_long, 
                                                             temp_parametros_cerrado=TEMP_PARAMETROS_CERRADO, 
                                                             temp_parametros_corrido=TEMP_PARAMETROS_CORRIDO, 
                                                             df_list=df_geo_list, 
                                                             umbral=10000)

In [77]:
# Conteo cerrado
ST_CONTEO_CERRADO = ST_CONTEO_CERRADO.reset_index()
ST_CONTEO_CERRADO = ST_CONTEO_CERRADO[['AGRUPACION', 'UNIDAD', 'CONTEO_T_1', 'CONTEO_T']]
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (ST_CONTEO_CERRADO['CONTEO_T_1'] == 0) & (ST_CONTEO_CERRADO['CONTEO_T'] == 0)
ST_CONTEO_CERRADO = ST_CONTEO_CERRADO[~condicion_1].reset_index(drop=True)


In [78]:
# Conteo corrido
ST_CONTEO_CORRIDO = ST_CONTEO_CORRIDO.reset_index()
ST_CONTEO_CORRIDO = ST_CONTEO_CORRIDO[['AGRUPACION', 'UNIDAD', 'CONTEO_T_1', 'CONTEO_T']]
# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (ST_CONTEO_CORRIDO['CONTEO_T_1'] == 0) & (ST_CONTEO_CORRIDO['CONTEO_T'] == 0)
ST_CONTEO_CORRIDO = ST_CONTEO_CORRIDO[~condicion_1].reset_index(drop=True)

## 5.5 Oportunidades

In [79]:
# Solo se debe cambiar la ubicación del archivo
path_insumos = 'C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/EXPORTACIONES/'
oportunidades_file = 'Oportunidades.xlsx'
dict_file = 'Diccionario-Departamentos-Oportunidades.xlsx'

In [80]:
# Importar datos
df_oportunidades = pd.read_excel(path_insumos + oportunidades_file, sheet_name="Base_proyectos", dtype=str)

In [81]:
df_oportunidades.columns

Index(['Eje', 'Apuesta', 'Cadena', 'Sector', 'Subsector', 'Producto', 'HUB',
       'Pais', 'Departamento', 'Continente'],
      dtype='object')

In [82]:
# Cambiar nombres para snowflake
df_oportunidades = df_oportunidades.rename(columns={
    'Eje' : 'EJE', 
    'Apuesta' : 'APUESTA',
    'Cadena' :'CADENA', 
    'Sector': 'SECTOR', 
    'Subsector': 'SUBSECTOR', 
    'Producto' : 'PRODUCTO', 
    'HUB': 'HUB',
    'Pais' : 'PAIS',
    'Departamento' : 'DEPARTAMENTO',
    'Continente' : 'CONTINENTE'
})

In [83]:
# Importar datos de nombres
df_dict_nombres = pd.read_excel(path_insumos + dict_file, sheet_name="Hoja1", dtype=str)

In [84]:
# Agregar códigos de nombres departamentos
df_oportunidades = pd.merge(df_oportunidades, df_dict_nombres, on=['DEPARTAMENTO'], how='left')
# Cambiar nombres 
df_oportunidades = df_oportunidades.rename(columns={
    'COD_DIVIPOLA' : 'COD_DIVIPOLA_DEPARTAMENTO'
})

In [85]:
# Confirmar que todo cruza
df_oportunidades[df_oportunidades['PAIS'].isnull()]['COD_DIVIPOLA_DEPARTAMENTO'].unique()

array([], dtype=object)

## 6. Balanza Comercial

In [86]:
# Solo se debe cambiar la ubicación del archivo
path_insumos = 'C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/EXPORTACIONES/'
balanza_file = 'Balanza Comercial.xlsx'

In [87]:
# Importar datos
df_importaciones = pd.read_excel(path_insumos + balanza_file, sheet_name="IMPORTACIONES", dtype= {'PAIS': str,
                                                                                                  'AÑO': str,
                                                                                                  'TIPO': str,
                                                                                                  'FOB_USD': float})
df_exportaciones = pd.read_excel(path_insumos + balanza_file, sheet_name="EXPORTACIONES", dtype= {'PAIS': str,
                                                                                                  'AÑO': str,
                                                                                                  'TIPO': str,
                                                                                                  'FOB_USD': float})

# Cambiar nombres correctamente
df_importaciones = df_importaciones.rename(columns={'PAIS': 'PAIS', 'AÑO': 'YEAR', 'TIPO': 'TIPO', 'FOB_USD': 'IMPORTACIONES_FOB'})
df_exportaciones = df_exportaciones.rename(columns={'PAIS': 'PAIS', 'AÑO': 'YEAR', 'TIPO': 'TIPO', 'FOB_USD': 'EXPORTACIONES_FOB'})

In [88]:
# Crear totales

# Exportaciones
df_total_exportaciones = df_exportaciones.groupby(['PAIS', 'YEAR'])[['EXPORTACIONES_FOB']].sum().reset_index()
# Agrega tipo
df_total_exportaciones['TIPO'] = 'Total'
# Ordenar columnas
df_total_exportaciones = df_total_exportaciones[['PAIS', 'YEAR', 'TIPO', 'EXPORTACIONES_FOB']]
# Agregar al dataframe por país
df_exportaciones = pd.concat([df_exportaciones, df_total_exportaciones])

In [89]:
# Crear totales

# Importaciones
df_total_importaciones = df_importaciones.groupby(['PAIS', 'YEAR'])[['IMPORTACIONES_FOB']].sum().reset_index()
# Agrega tipo
df_total_importaciones['TIPO'] = 'Total'
# Ordenar columnas
df_total_importaciones = df_total_importaciones[['PAIS', 'YEAR', 'TIPO', 'IMPORTACIONES_FOB']]
# Agregar al dataframe por país
df_importaciones = pd.concat([df_importaciones, df_total_importaciones])

In [90]:
# Crear balanza comercial Colombia
total_impo = df_importaciones.groupby(['TIPO', 'YEAR'])[['IMPORTACIONES_FOB']].sum().reset_index()
total_expo = df_exportaciones.groupby(['TIPO', 'YEAR'])[['EXPORTACIONES_FOB']].sum().reset_index()

# Crear agregado de totales
df_total_balanza = pd.merge(total_impo, total_expo, on=['TIPO', 'YEAR'], how='left')

# País Colombia total
df_total_balanza['PAIS'] = 'COLOMBIA'

# Ordenar columnas
df_total_balanza = df_total_balanza[['PAIS', 'YEAR', 'TIPO', 'IMPORTACIONES_FOB', 'EXPORTACIONES_FOB']]

In [91]:
# Crear base por país
df_balanza_insumo = pd.merge(df_importaciones, df_exportaciones, on=['PAIS', 'TIPO', 'YEAR'], how='outer')

In [92]:
# Agregar suma total de balanza de Colombia
df_balanza_insumo = pd.concat([df_balanza_insumo, df_total_balanza])

In [93]:
# Rellenar los campos 'IMPORTACIONES_FOB' y 'EXPORTACIONES_FOB' con 0 donde haya NaN
df_balanza_insumo[['IMPORTACIONES_FOB', 'EXPORTACIONES_FOB']] = df_balanza_insumo[['IMPORTACIONES_FOB', 'EXPORTACIONES_FOB']].fillna(0)

In [94]:
# Calcular balanza
df_balanza_insumo['BALANZA'] = df_balanza_insumo['EXPORTACIONES_FOB'] - df_balanza_insumo['IMPORTACIONES_FOB']

In [95]:
# Elegir columnas 
df_balanza_final = df_balanza_insumo[['PAIS', 'YEAR', 'TIPO', 'BALANZA']]

## 7. Tablas TLC

In [96]:
# Se trae la tabla de tlcs con la llave de expotaciones
query_tlcs = """
SELECT *
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.TLCS_TABLA AS A
ORDER BY A.PAIS_LLAVE_EXPORTACIONES ASC;
"""
# Ejecutar
df_paises_tlcs = snow_func.snowflake_sql(conn, query_tlcs)

#### 7.1 Colombia

In [97]:
# Traer datos cerrados de exportaciones
df_insumo_cerrado_colombia = DF_ST_CATEGORIAS_CERRADO.loc[(DF_ST_CATEGORIAS_CERRADO['AGRUPACION'] == 'PAISES') &
                             (DF_ST_CATEGORIAS_CERRADO['TABLA'] == 'TIPOS') &
                             (DF_ST_CATEGORIAS_CERRADO['CATEGORIA'] == 'No Mineras'),
                            ['UNIDAD', 'SUMA_USD_T_1', 'SUMA_USD_T']]

# Traer datos corridos de exportaciones
df_insumo_corrido_colombia = DF_ST_CATEGORIAS_CORRIDO.loc[(DF_ST_CATEGORIAS_CORRIDO['AGRUPACION'] == 'PAISES') &
                             (DF_ST_CATEGORIAS_CORRIDO['TABLA'] == 'TIPOS') &
                             (DF_ST_CATEGORIAS_CORRIDO['CATEGORIA'] == 'No Mineras'),
                            ['UNIDAD', 'SUMA_USD_T_1', 'SUMA_USD_T']]

In [98]:
# Procedimiento para cerrado

# Unir valores exportados a los países definidos por TLCs en la tabla relacional
df_tlcs_colombia_cerrado = df_paises_tlcs.merge(df_insumo_cerrado_colombia, how='left', left_on='PAIS_LLAVE_EXPORTACIONES', right_on='UNIDAD')

# Crear las sumas por TLC
df_tlcs_colombia_cerrado = df_tlcs_colombia_cerrado.groupby('NOMBRE_TLC', as_index=False)[['SUMA_USD_T_1', 'SUMA_USD_T']].sum()

# Crear columnas extras
df_tlcs_colombia_cerrado['AGRUPACION'] = 'COLOMBIA'
df_tlcs_colombia_cerrado['UNIDAD'] = 'COLOMBIA'
df_tlcs_colombia_cerrado['TABLA'] = 'TLCS'

# Agregar columna de diferencia absoluta y porcentual
# Absoluta
df_tlcs_colombia_cerrado['DIFERENCIA_ABSOLUTA'] = df_tlcs_colombia_cerrado['SUMA_USD_T'] - df_tlcs_colombia_cerrado['SUMA_USD_T_1']
# Porcentual
df_tlcs_colombia_cerrado['DIFERENCIA_PORCENTUAL'] = np.where(
    df_tlcs_colombia_cerrado['SUMA_USD_T_1'] == 0,
    np.where(df_tlcs_colombia_cerrado['SUMA_USD_T'] > 0, 100, np.where(df_tlcs_colombia_cerrado['SUMA_USD_T'] == 0, 0, -100)),
    (df_tlcs_colombia_cerrado['DIFERENCIA_ABSOLUTA'] / df_tlcs_colombia_cerrado['SUMA_USD_T_1']) * 100
)

# Cambiar nombre de TLC a categoria para mantener consistencia
df_tlcs_colombia_cerrado = df_tlcs_colombia_cerrado.rename(columns={'NOMBRE_TLC': 'CATEGORIA'})

# Reorndear columnas
df_tlcs_colombia_cerrado = df_tlcs_colombia_cerrado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1','SUMA_USD_T', 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL']]

# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (df_tlcs_colombia_cerrado['SUMA_USD_T_1'] == 0) & (df_tlcs_colombia_cerrado['SUMA_USD_T'] == 0)
df_tlcs_colombia_cerrado = df_tlcs_colombia_cerrado[~condicion_1].reset_index(drop=True)

In [99]:
# Agregar datos a tabla ST_CATEGORIAS_CERRADO
DF_ST_CATEGORIAS_CERRADO = pd.concat([DF_ST_CATEGORIAS_CERRADO, df_tlcs_colombia_cerrado])

In [100]:
# Procedimiento para corrido

# Unir valores exportados a los países definidos por TLCs en la tabla relacional
df_tlcs_colombia_corrido = df_paises_tlcs.merge(df_insumo_corrido_colombia, how='left', left_on='PAIS_LLAVE_EXPORTACIONES', right_on='UNIDAD')

# Crear las sumas por TLC
df_tlcs_colombia_corrido = df_tlcs_colombia_corrido.groupby('NOMBRE_TLC', as_index=False)[['SUMA_USD_T_1', 'SUMA_USD_T']].sum()

# Crear columnas extras
df_tlcs_colombia_corrido['AGRUPACION'] = 'COLOMBIA'
df_tlcs_colombia_corrido['UNIDAD'] = 'COLOMBIA'
df_tlcs_colombia_corrido['TABLA'] = 'TLCS'

# Agregar columna de diferencia absoluta y porcentual
# Absoluta
df_tlcs_colombia_corrido['DIFERENCIA_ABSOLUTA'] = df_tlcs_colombia_corrido['SUMA_USD_T'] - df_tlcs_colombia_corrido['SUMA_USD_T_1']
# Porcentual
df_tlcs_colombia_corrido['DIFERENCIA_PORCENTUAL'] = np.where(
    df_tlcs_colombia_corrido['SUMA_USD_T_1'] == 0,
    np.where(df_tlcs_colombia_corrido['SUMA_USD_T'] > 0, 100, np.where(df_tlcs_colombia_corrido['SUMA_USD_T'] == 0, 0, -100)),
    (df_tlcs_colombia_corrido['DIFERENCIA_ABSOLUTA'] / df_tlcs_colombia_corrido['SUMA_USD_T_1']) * 100
)

# Cambiar nombre de TLC a categoria para mantener consistencia
df_tlcs_colombia_corrido = df_tlcs_colombia_corrido.rename(columns={'NOMBRE_TLC': 'CATEGORIA'})

# Reorndear columnas
df_tlcs_colombia_corrido = df_tlcs_colombia_corrido[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1','SUMA_USD_T', 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL']]

# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (df_tlcs_colombia_corrido['SUMA_USD_T_1'] == 0) & (df_tlcs_colombia_corrido['SUMA_USD_T'] == 0)
df_tlcs_colombia_corrido = df_tlcs_colombia_corrido[~condicion_1].reset_index(drop=True)

In [101]:
# Agregar datos a tabla ST_CATEGORIAS_CORRIDO
DF_ST_CATEGORIAS_CORRIDO = pd.concat([DF_ST_CATEGORIAS_CORRIDO, df_tlcs_colombia_corrido])

#### 7.2 Departamentos

In [102]:
# Traer datos cerrados de exportaciones
df_insumo_cerrado_departamento = DF_ST_CATEGORIAS_CERRADO.loc[(DF_ST_CATEGORIAS_CERRADO['AGRUPACION'] == 'DEPARTAMENTOS') &
                             (DF_ST_CATEGORIAS_CERRADO['TABLA'] == 'PAIS'),
                            ['UNIDAD', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T']]

# Traer datos corridos de exportaciones
df_insumo_corrido_departamento = DF_ST_CATEGORIAS_CORRIDO.loc[(DF_ST_CATEGORIAS_CORRIDO['AGRUPACION'] == 'DEPARTAMENTOS') &
                             (DF_ST_CATEGORIAS_CORRIDO['TABLA'] == 'PAIS'),
                            ['UNIDAD', 'CATEGORIA', 'SUMA_USD_T_1', 'SUMA_USD_T']]

# Crear df con todas la combinaciones posibles de departamentos

# Cerrado
df_departamentos_cerrado = DF_ST_CATEGORIAS_CERRADO.loc[(DF_ST_CATEGORIAS_CERRADO['AGRUPACION'] == 'DEPARTAMENTOS') &
                             (DF_ST_CATEGORIAS_CERRADO['TABLA'] == 'PAIS'),
                            ['AGRUPACION', 'UNIDAD']].drop_duplicates()

# Corrido
df_departamentos_corrido = DF_ST_CATEGORIAS_CORRIDO.loc[(DF_ST_CATEGORIAS_CORRIDO['AGRUPACION'] == 'DEPARTAMENTOS') &
                             (DF_ST_CATEGORIAS_CORRIDO['TABLA'] == 'PAIS'),
                            ['AGRUPACION', 'UNIDAD']].drop_duplicates()

# Opciones totales
df_departamentos = pd.concat([df_departamentos_cerrado, df_departamentos_corrido]).drop_duplicates()

# Agregar combinaciones de tlcs
df_departamentos = df_departamentos.merge(df_paises_tlcs, how='cross')

In [103]:
# Procedimiento para cerrado

# Unir valores exportados a los departamentos definidos
df_tlcs_departamentos_cerrado = df_departamentos.merge(df_insumo_cerrado_departamento, how='left', left_on=['UNIDAD', 'PAIS_LLAVE_EXPORTACIONES'], right_on=['UNIDAD', 'CATEGORIA'])

# Crear las sumas por TLC
df_tlcs_departamentos_cerrado = df_tlcs_departamentos_cerrado.groupby(['AGRUPACION', 'UNIDAD', 'NOMBRE_TLC'], as_index=False)[['SUMA_USD_T_1', 'SUMA_USD_T']].sum()

# Crear columnas extras
df_tlcs_departamentos_cerrado['TABLA'] = 'TLCS'

# Agregar columna de diferencia absoluta y porcentual
# Absoluta
df_tlcs_departamentos_cerrado['DIFERENCIA_ABSOLUTA'] = df_tlcs_departamentos_cerrado['SUMA_USD_T'] - df_tlcs_departamentos_cerrado['SUMA_USD_T_1']
# Porcentual
df_tlcs_departamentos_cerrado['DIFERENCIA_PORCENTUAL'] = np.where(
    df_tlcs_departamentos_cerrado['SUMA_USD_T_1'] == 0,
    np.where(df_tlcs_departamentos_cerrado['SUMA_USD_T'] > 0, 100, np.where(df_tlcs_departamentos_cerrado['SUMA_USD_T'] == 0, 0, -100)),
    (df_tlcs_departamentos_cerrado['DIFERENCIA_ABSOLUTA'] / df_tlcs_departamentos_cerrado['SUMA_USD_T_1']) * 100
)

# Cambiar nombre de TLC a categoria para mantener consistencia
df_tlcs_departamentos_cerrado = df_tlcs_departamentos_cerrado.rename(columns={'NOMBRE_TLC': 'CATEGORIA'})

# Reorndear columnas
df_tlcs_departamentos_cerrado = df_tlcs_departamentos_cerrado[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1','SUMA_USD_T', 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL']]

# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (df_tlcs_departamentos_cerrado['SUMA_USD_T_1'] == 0) & (df_tlcs_departamentos_cerrado['SUMA_USD_T'] == 0)
df_tlcs_departamentos_cerrado = df_tlcs_departamentos_cerrado[~condicion_1].reset_index(drop=True)

In [104]:
# Agregar datos a tabla ST_CATEGORIAS_CERRADO
DF_ST_CATEGORIAS_CERRADO = pd.concat([DF_ST_CATEGORIAS_CERRADO, df_tlcs_departamentos_cerrado])

In [105]:
# Procedimiento para corrido

# Unir valores exportados a los departamentos definidos
df_tlcs_departamentos_corrido = df_departamentos.merge(df_insumo_corrido_departamento, how='left', left_on=['UNIDAD', 'PAIS_LLAVE_EXPORTACIONES'], right_on=['UNIDAD', 'CATEGORIA'])

# Crear las sumas por TLC
df_tlcs_departamentos_corrido = df_tlcs_departamentos_corrido.groupby(['AGRUPACION', 'UNIDAD', 'NOMBRE_TLC'], as_index=False)[['SUMA_USD_T_1', 'SUMA_USD_T']].sum()

# Crear columnas extras
df_tlcs_departamentos_corrido['TABLA'] = 'TLCS'

# Agregar columna de diferencia absoluta y porcentual
# Absoluta
df_tlcs_departamentos_corrido['DIFERENCIA_ABSOLUTA'] = df_tlcs_departamentos_corrido['SUMA_USD_T'] - df_tlcs_departamentos_corrido['SUMA_USD_T_1']
# Porcentual
df_tlcs_departamentos_corrido['DIFERENCIA_PORCENTUAL'] = np.where(
    df_tlcs_departamentos_corrido['SUMA_USD_T_1'] == 0,
    np.where(df_tlcs_departamentos_corrido['SUMA_USD_T'] > 0, 100, np.where(df_tlcs_departamentos_corrido['SUMA_USD_T'] == 0, 0, -100)),
    (df_tlcs_departamentos_corrido['DIFERENCIA_ABSOLUTA'] / df_tlcs_departamentos_corrido['SUMA_USD_T_1']) * 100
)

# Cambiar nombre de TLC a categoria para mantener consistencia
df_tlcs_departamentos_corrido = df_tlcs_departamentos_corrido.rename(columns={'NOMBRE_TLC': 'CATEGORIA'})

# Reorndear columnas
df_tlcs_departamentos_corrido = df_tlcs_departamentos_corrido[['AGRUPACION', 'UNIDAD', 'TABLA', 'CATEGORIA', 'SUMA_USD_T_1','SUMA_USD_T', 'DIFERENCIA_ABSOLUTA', 'DIFERENCIA_PORCENTUAL']]

# Eliminar filas donde los valores en ambas columnas de sumas sean cero
condicion_1 = (df_tlcs_departamentos_corrido['SUMA_USD_T_1'] == 0) & (df_tlcs_departamentos_corrido['SUMA_USD_T'] == 0)
df_tlcs_departamentos_corrido = df_tlcs_departamentos_corrido[~condicion_1].reset_index(drop=True)

In [106]:
# Agregar datos a tabla ST_CATEGORIAS_CORRIDO
DF_ST_CATEGORIAS_CORRIDO = pd.concat([DF_ST_CATEGORIAS_CORRIDO, df_tlcs_departamentos_corrido])

## 8. Crear base de datos de validación

In [112]:
# Definir los nombres de las columnas y agrupaciones
columna_base_exportaciones = ['Continente', 'Pais Destino', 'HUB', 'Departamento Origen']
agrupacion_base_exportaciones = ['CONTINENTES', 'PAISES', 'HUBS', 'DEPARTAMENTOS']

# Columnas a sumar
columnas_cerrado = {'USD': ['2022 USD', '2023 USD'], 'KG': ['2022 KG Neto', '2023 KG Neto']}
columnas_corrido = {'USD': ['2023 USD (Ene-Oct)', '2024 USD (Ene-Oct)'], 'KG': ['2023 KG Neto (Ene-Oct)', '2024 KG Neto (Ene-Oct)']}

# Crear diccionario para la validación
# Diccionario con las columnas para cada tipo de período y medida
columnas_dict = {
    'CERRADO': {
        'USD': ['2022 USD', '2023 USD'],
        'KG': ['2022 KG Neto', '2023 KG Neto']
    },
    'CORRIDO': {
        'USD': ['2023 USD (Ene-Oct)', '2024 USD (Ene-Oct)'],
        'KG': ['2023 KG Neto (Ene-Oct)', '2024 KG Neto (Ene-Oct)']
    }
}

# Especificar las medidas y tipos de periodo a considerar
medidas = ['USD', 'KG']

In [113]:
def generar_validacion(df, columnas, medida, tipo_periodo):
    """
    Genera un DataFrame de validación para los totales de exportaciones agrupados por una columna base.

    Args:
        df (pd.DataFrame): DataFrame fuente con los datos de exportaciones.
        columnas (list): Lista de dos columnas numéricas a sumar (por ejemplo, USD o KG).
        medida (str): Unidad de medida, como 'USD' o 'KG'.
        tipo_periodo (str): Tipo de periodo, como 'CERRADO' o 'CORRIDO'.

    Returns:
        pd.DataFrame: DataFrame con las exportaciones agrupadas y columnas renombradas:
            - AGRUPACION: Tipo de agrupación (CONTINENTES, PAISES, etc.).
            - UNIDAD: Nombre de la unidad de agrupación.
            - MEDIDA: Unidad de medida (USD o KG).
            - TIPO_PERIODO: Tipo de periodo (CERRADO o CORRIDO).
            - SUMA_T_1: Suma del primer año/periodo.
            - SUMA_T: Suma del segundo año/periodo.
    """
    # Inicializar un DataFrame vacío para almacenar los resultados
    df_validacion = pd.DataFrame()

    # Iterar sobre las columnas base de exportaciones y sus agrupaciones
    for columna, agrupacion in zip(columna_base_exportaciones, agrupacion_base_exportaciones):
        # Agrupar el DataFrame fuente por la columna base y sumar las columnas especificadas
        df_agrupado = df.groupby([columna])[columnas].sum().reset_index()

        # Renombrar columnas de las sumas
        df_agrupado.rename(columns={columnas[0]: 'SUMA_T_1', columnas[1]: 'SUMA_T'}, inplace=True)
        
        # Renombrar la columna base a 'UNIDAD' para estandarización
        df_agrupado.rename(columns={columna: 'UNIDAD'}, inplace=True)
        
        # Agregar las columnas adicionales al DataFrame agrupado
        df_agrupado['AGRUPACION'] = agrupacion
        df_agrupado['MEDIDA'] = medida
        df_agrupado['TIPO_PERIODO'] = tipo_periodo

        # Reorganizar las columnas
        df_agrupado = df_agrupado[['AGRUPACION', 'UNIDAD', 'MEDIDA', 'TIPO_PERIODO', 'SUMA_T_1', 'SUMA_T']]

        # Concatenar los resultados al DataFrame de validación
        df_validacion = pd.concat([df_validacion, df_agrupado], ignore_index=True)

    return df_validacion


def generar_validacion_colombia(df, columnas, medida, tipo_periodo):
    """
    Genera un DataFrame de validación para Colombia, calculando los totales de las columnas numéricas.

    Args:
        df (pd.DataFrame): DataFrame fuente con los datos de exportaciones.
        columnas (list): Lista de dos columnas numéricas a sumar (por ejemplo, USD o KG).
        medida (str): Unidad de medida, como 'USD' o 'KG'.
        tipo_periodo (str): Tipo de periodo, como 'CERRADO' o 'CORRIDO'.

    Returns:
        pd.DataFrame: DataFrame con los totales de exportaciones para Colombia, incluyendo:
            - AGRUPACION: "COLOMBIA".
            - UNIDAD: "COLOMBIA".
            - MEDIDA: Unidad de medida (USD o KG).
            - TIPO_PERIODO: Tipo de periodo (CERRADO o CORRIDO).
            - SUMA_T_1: Suma del primer año/periodo.
            - SUMA_T: Suma del segundo año/periodo.
    """
    # Calcular los totales directamente
    totales = df[columnas].sum()

    # Crear un DataFrame con los totales y columnas adicionales
    totales_df = pd.DataFrame([{
        'AGRUPACION': 'COLOMBIA',
        'UNIDAD': 'COLOMBIA',
        'MEDIDA': medida,
        'TIPO_PERIODO': tipo_periodo,
        'SUMA_T_1': totales[columnas[0]],
        'SUMA_T': totales[columnas[1]]
    }])

    return totales_df


def generar_validaciones(df, columnas_dict, medidas, tipos_periodo, incluir_colombia=False):
    """
    Genera un DataFrame consolidado de validación a partir de diferentes combinaciones
    de columnas, medidas y períodos.

    Args:
        df (pd.DataFrame): DataFrame de entrada con los datos base.
        columnas_dict (dict): Diccionario con claves para tipos de período y medidas.
        medidas (list): Lista de medidas, como ['USD', 'KG'].
        tipos_periodo (list): Lista de tipos de período (pueden incluir componentes adicionales).
                              Ejemplo: ['CERRADO-TOTAL-EXPORTADO', 'CORRIDO-TOTAL-EXPORTADO'].
        incluir_colombia (bool): Si se incluye la validación para el total de Colombia.

    Returns:
        pd.DataFrame: DataFrame consolidado con todas las validaciones generadas.
    """
    df_validacion_final = pd.DataFrame()

    for tipo_periodo in tipos_periodo:
        # Extraer la clave del período para buscar en columnas_dict
        periodo_clave = tipo_periodo.split('-')[0]  # Ejemplo: 'CERRADO' de 'CERRADO-TOTAL-EXPORTADO'

        for medida in medidas:
            try:
                # Obtener las columnas según el período y medida
                columnas = columnas_dict[periodo_clave][medida]

                # Validación por agrupaciones generales
                df_validacion_general = generar_validacion(df, columnas, medida, tipo_periodo)
                df_validacion_final = pd.concat([df_validacion_final, df_validacion_general], ignore_index=True)

                # Validación total para Colombia (si aplica)
                if incluir_colombia:
                    df_validacion_colombia = generar_validacion_colombia(df, columnas, medida, tipo_periodo)
                    df_validacion_final = pd.concat([df_validacion_final, df_validacion_colombia], ignore_index=True)
            except KeyError:
                raise KeyError(f"El período '{periodo_clave}' o la medida '{medida}' no existe en columnas_dict.")

    return df_validacion_final

In [114]:
# Crear totales de control para 'CONTINENTES', 'PAISES', 'HUBS', 'DEPARTAMENTOS', 'COLOMBIA'

#####################################################
# TOTAL EXPORTADO USD Y KG: AÑO CERRADO Y AÑO CORRIDO
#####################################################

# Validación de total exportado para USD y KG: CERRADO Y CORRIDO
tipos_periodo_total = ['CERRADO-TOTAL-EXPORTADO', 'CORRIDO-TOTAL-EXPORTADO']

# Generar validaciones
df_validacion_total_exportado = generar_validaciones(
    df=df_insumo_validacion,
    columnas_dict=columnas_dict,
    medidas=medidas,
    tipos_periodo=tipos_periodo_total,
    incluir_colombia=True  # Incluye totales para Colombia
)

###############################################################
# TOTAL EXPORTADO NO MINERO USD Y KG: AÑO CERRADO Y AÑO CORRIDO
###############################################################

df_nme = df_insumo_validacion[df_insumo_validacion['Tipo']=='No Mineras']

# Validación de NME exportado para USD y KG: CERRADO Y CORRIDO
tipos_periodo_nme = ['CERRADO-NME-EXPORTADO', 'CORRIDO-NME-EXPORTADO']

# Generar validaciones
df_validacion_nme_exportado = generar_validaciones(
    df=df_nme,
    columnas_dict=columnas_dict,
    medidas=medidas,
    tipos_periodo=tipos_periodo_nme,
    incluir_colombia=True  # Incluye totales para Colombia
)

############################################################
# TOTAL EXPORTADO MINERO USD Y KG: AÑO CERRADO Y AÑO CORRIDO
############################################################

df_minero = df_insumo_validacion[df_insumo_validacion['Tipo']=='Mineras']

# Validación de mineros exportado para USD y KG: CERRADO Y CORRIDO
tipos_periodo_mineros = ['CERRADO-MINERO-EXPORTADO', 'CORRIDO-MINERO-EXPORTADO']

# Generar validaciones
df_validacion_minero_exportado = generar_validaciones(
    df=df_minero,
    columnas_dict=columnas_dict,
    medidas=medidas,
    tipos_periodo=tipos_periodo_mineros,
    incluir_colombia=True  # Incluye totales para Colombia
)

#################################
# CREAR DF CON EL RESULTADO FINAL
#################################

DF_EXPORTACIONES_CONTROL = pd.concat([df_validacion_total_exportado, df_validacion_nme_exportado, df_validacion_minero_exportado])

In [115]:
# Correlativa con la definición de TLCs
query_tlcs = """
SELECT DISTINCT A.PAIS_LLAVE_EXPORTACIONES,
	A.NOMBRE_TLC
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
WHERE A.PAIS_LLAVE_EXPORTACIONES IS NOT NULL
ORDER BY 1 ASC;
"""
# Ejecutar
df_tlcs = snow_func.snowflake_sql(conn, query_tlcs)

In [116]:
# Unir con la correlativa de TLCs
df_insumo_tlcs = DF_EXPORTACIONES_CONTROL.merge(df_tlcs, how='left', left_on=['UNIDAD'], right_on=['PAIS_LLAVE_EXPORTACIONES'])

# Generar las sumas agregadas por TLCs
df_insumo_tlcs = df_insumo_tlcs.groupby(['MEDIDA', 'TIPO_PERIODO', 'NOMBRE_TLC'])[['SUMA_T_1', 'SUMA_T']].sum().reset_index()

# Renombrar la columna correlativa seleccionada a 'UNIDAD' para estandarizar los nombres de las columnas.
df_insumo_tlcs = df_insumo_tlcs.rename(columns={'NOMBRE_TLC': 'UNIDAD'})

# Añadir la columna 'AGRUPACION' 
df_insumo_tlcs['AGRUPACION'] = 'TLCS'

# Reorganizar las columnas
DF_EXPORTACIONES_CONTROL_TLCS = df_insumo_tlcs[['AGRUPACION', 'UNIDAD', 'MEDIDA', 'TIPO_PERIODO', 'SUMA_T_1', 'SUMA_T']]

In [117]:
# Concatenar ambos resultado
DF_EXPORTACIONES_CONTROL = pd.concat([DF_EXPORTACIONES_CONTROL, DF_EXPORTACIONES_CONTROL_TLCS])

## 9. Subir a Snowflake

In [118]:
# Usar base de datos:
sql_database_usar = """
USE DOCUMENTOS_COLOMBIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_database_usar)

,status
0,Statement executed successfully.


In [119]:
# Usar esquema exportaciones:
sql_schema_exportaciones = """
USE SCHEMA EXPORTACIONES;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_exportaciones)

,status
0,Statement executed successfully.


In [120]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,EXPORTACIONES


In [121]:
# Lista de pd a subir
bases_de_datos = [
    DF_ST_CATEGORIAS_CERRADO, 
    DF_ST_CATEGORIAS_CORRIDO, 
    DF_ST_CATEGORIAS_PESO_CERRADO, 
    DF_ST_CATEGORIAS_PESO_CORRIDO, 
    DF_ST_NIT_CERRADO, 
    DF_ST_NIT_CORRIDO, 
    ST_CONTEO_CERRADO, 
    ST_CONTEO_CORRIDO, 
    df_oportunidades,
    df_balanza_final,
    DF_EXPORTACIONES_CONTROL
]

nombres_tablas = [
    'ST_CATEGORIAS_CERRADO', 
    'ST_CATEGORIAS_CORRIDO', 
    'ST_CATEGORIAS_PESO_CERRADO', 
    'ST_CATEGORIAS_PESO_CORRIDO', 
    'ST_NIT_CERRADO', 
    'ST_NIT_CORRIDO', 
    'ST_CONTEO_CERRADO', 
    'ST_CONTEO_CORRIDO', 
    'OPORTUNIDADES',
    'BALANZA',
    'DF_EXPORTACIONES_CONTROL'
]

In [122]:
# pd de verificación
df_resultados_verificacion = pd.DataFrame()
# Subir y verificar bases a Snowflake
for base, tabla in zip(bases_de_datos, nombres_tablas):
    # Cargar el DataFrame en Snowflake y capturar el mensaje de carga
    mensaje_carga = snow_func.snowflake_cargar_df(conn, base, f'{tabla}')
    
    # Verificar y almacenar el resultado en el DataFrame
    resultado = snow_func.snowflake_sql(conn, f"SELECT COUNT(*) FROM {tabla};")
    total_registros = resultado  # Extraer el total de registros

    # Crear un DataFrame temporal para la nueva fila
    nueva_fila = pd.DataFrame({
        'Tabla': [tabla],
        'Total_Registros': [total_registros],
        'Mensaje_Carga': [mensaje_carga]
    })

    # Concatenar la nueva fila al DataFrame de resultados
    df_resultados_verificacion = pd.concat([df_resultados_verificacion, nueva_fila], ignore_index=True)

In [123]:
# Ver resultados
df_resultados_verificacion

,Tabla,Total_Registros,Mensaje_Carga
0,ST_CATEGORIAS_CERRADO,COUNT(*) 0 28190,DataFrame cargado exitosamente en la tabla: 28190 filas en 1 chunks.\nTiempo de carga: 4.66 segundos.\nProceso terminado
1,ST_CATEGORIAS_CORRIDO,COUNT(*) 0 27725,DataFrame cargado exitosamente en la tabla: 27725 filas en 1 chunks.\nTiempo de carga: 2.56 segundos.\nProceso terminado
2,ST_CATEGORIAS_PESO_CERRADO,COUNT(*) 0 5550,DataFrame cargado exitosamente en la tabla: 5550 filas en 1 chunks.\nTiempo de carga: 2.88 segundos.\nProceso terminado
3,ST_CATEGORIAS_PESO_CORRIDO,COUNT(*) 0 5423,DataFrame cargado exitosamente en la tabla: 5423 filas en 1 chunks.\nTiempo de carga: 2.32 segundos.\nProceso terminado
4,ST_NIT_CERRADO,COUNT(*) 0 121919,DataFrame cargado exitosamente en la tabla: 121919 filas en 1 chunks.\nTiempo de carga: 3.52 segundos.\nProceso terminado
5,ST_NIT_CORRIDO,COUNT(*) 0 116671,DataFrame cargado exitosamente en la tabla: 116671 filas en 1 chunks.\nTiempo de carga: 3.68 segundos.\nProceso terminado
6,ST_CONTEO_CERRADO,COUNT(*) 0 250,DataFrame cargado exitosamente en la tabla: 250 filas en 1 chunks.\nTiempo de carga: 1.79 segundos.\nProceso terminado
7,ST_CONTEO_CORRIDO,COUNT(*) 0 250,DataFrame cargado exitosamente en la tabla: 250 filas en 1 chunks.\nTiempo de carga: 1.97 segundos.\nProceso terminado
8,OPORTUNIDADES,COUNT(*) 0 15957,DataFrame cargado exitosamente en la tabla: 15957 filas en 1 chunks.\nTiempo de carga: 2.11 segundos.\nProceso terminado
9,BALANZA,COUNT(*) 0 2594,DataFrame cargado exitosamente en la tabla: 2594 filas en 1 chunks.\nTiempo de carga: 2.19 segundos.\nProceso terminado


### Cerrar sesión, conexión y cursor

In [124]:
conn.close()
session.close()